In [1]:
import json
import os
import re
import collections
import numpy as np
import pandas as pd
import time
import tensorflow as tf
import LAC
from sklearn.feature_selection import RFE
import happybase
from sklearn.cluster import KMeans 
import joblib
import sklearn
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from bayes_opt import BayesianOptimization
from sklearn.model_selection import cross_val_score
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 
tf.config.set_visible_devices([], 'GPU')

In [2]:
sklearn.__version__

'0.22.2'

In [4]:
df_list = []
with open("../data/predict_user_attribute20200911/raw_data/user_profile/trade_data") as f:
    for line in f:
        df_list.append(line.strip().split('\001'))
df = pd.DataFrame(df_list)

In [5]:
df[1] = df[1].apply(lambda x : x.lower())

In [6]:
all_counter = collections.Counter([v for v in df[1]])

In [7]:
def temp_function(x):
    #x = datetime.datetime.now().year - int(x.split("-")[0])
    y = 0
    if x in set(['it','互联网','tmt从业人员','人工智能/前沿科技','企业服务/云计算/大数据']):
        y = "IT/移动互联网"
    elif x in set(['教育/专业服务/培训','内容/营销/传播','营销','贸易/零售批发','酒店/餐饮/旅游']):
        y = "零售消费"
    elif x in set(['金融','投资人']):
        y = "金融"
    elif x in set(['汽车/出行',]):
        y = "汽车"
    elif x in set(['电商/仓储物流','仓储物流','商业服务','商业服务-o2o/服务/社区']):
        y = "电商"
    elif x in set(['加工制造']):
        y = "制造业"
    elif x in set(['智能硬件/硬件制造']):
        y = "IOT"
    elif x in set(['文化/体育/娱乐业']):
        y = "游戏"
    elif x in set(['房地产',]):
        y = "房地产"
    elif x in set(['制药医疗/生物/卫生保健',]):
        y = "医疗"
    elif x in set(['创业者','政府/非盈利机构','法律','其他','媒体','能源相关','农业','学生','-1']):
        y = "其他"
    else:
        raise
    return y

In [8]:
df[1] = df[1].apply(lambda x : temp_function(x))

In [9]:
df[1].unique()

array(['零售消费', 'IT/移动互联网', '电商', '金融', '其他', '游戏', '制造业', '汽车', 'IOT',
       '房地产', '医疗'], dtype=object)

In [10]:
collections.Counter(df[1])

Counter({'零售消费': 933,
         'IT/移动互联网': 2036,
         '电商': 644,
         '金融': 1143,
         '其他': 1382,
         '游戏': 163,
         '制造业': 164,
         '汽车': 98,
         'IOT': 92,
         '房地产': 159,
         '医疗': 137})

In [11]:
df.columns = ["aid","trade",*["c{}".format(v) for v in range(10)]]
df

,aid,trade,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9
0,10113,零售消费,378020,378036,377965,377980,378047,378021,377903,377873,378033,378025
1,10140,IT/移动互联网,381349,381108,381299,381252,380842,381326,379187,381311,381059,380919
2,101966,电商,381295,381108,381284,381313,381106,380842,381316,381279,381315,381310
3,103927,金融,365636,365655,365723,365476,365546,365554,365423,365698,365331,365761
4,10714,IT/移动互联网,376192,376167,377536,376189,376126,374016,373949,378198,374017,379878
...,...,...,...,...,...,...,...,...,...,...,...,...
6946,8,零售消费,379195,379107,379194,380776,378982,379529,379133,379489,379526,379004
6947,83135,金融,374584,354849,377736,371117,377233,381026,379256,378084,379411,381309
6948,84249,IT/移动互联网,380858,379194,380797,380393,379607,380902,381078,379795,379977,378982
6949,89195,IT/移动互联网,358942,360290,360133,359550,368330,368145,363890,363980,367415,363881


In [12]:
article_dict = {}
with open("../data/predict_user_attribute20200911/raw_data/user_profile/title_tokenize.jsonl",encoding="utf8") as f:
    for line in f:
        line_dict = json.loads(line.strip())
        aid = line_dict["aid"]
        article_dict[aid] = {}
        article_dict[aid]["title_tokenize"] = line_dict["title_tokenize"]

In [13]:
text_list = []
y_list = []
for index in df.index:
    y = df.loc[index,"trade"]
    temp_text_list = []
    for i in range(10):
        c_text_list = []
        c = "c{}".format(i)
        aid = str(df.loc[index,c]).strip()
        if aid:
            if aid in article_dict:
                c_text_list = [*article_dict[aid]["title_tokenize"]]
            else:
                print("aid不存在",index,c,aid)
        else:
            print(index)
        temp_text_list.append(c_text_list)     
    text_list.append(temp_text_list)
    y_list.append(y)

aid不存在 55 c3 130
aid不存在 236 c8 161
aid不存在 290 c3 130
aid不存在 314 c8 142
aid不存在 912 c4 130
aid不存在 945 c6 134
aid不存在 1127 c1 108
aid不存在 2081 c2 130
aid不存在 2502 c5 130
aid不存在 3328 c8 126
aid不存在 3346 c3 130
aid不存在 3654 c5 130
aid不存在 3719 c2 213
aid不存在 5092 c1 130
aid不存在 6424 c3 130


In [14]:
with open(r"../data/predict_user_attribute20200911/raw_data/user_profile/trade/最终数据集tokenize_title.jsonl","w") as f:
    for tag,content_tokenize in zip(y_list,text_list):
        f.write(json.dumps({"tag":tag,"all_content_tokenize":content_tokenize},ensure_ascii=False) + "\n")

724

673

696

631

578

...

603

825

589

615

526

In [2]:
import tensorflow as tf

def build_model(
    input_number:int,
    sentence_maxlen:int,
    vocab_size:int,
    tag_size:int,
    embedding_dim:int,
    embedding_matrix=None,
    is_embedding_training:bool=True,
    embedding_dropout_rate:float = 0.0,
    learning_rate = 1e-3,
    ):
    """建立模型
    input:
        sentence_maxlen : 句子的长度
        vocab_size : 词的个数
        tag_size : 分类的个数
        embedding_dim : word2vec训练时设置的向量长度
        embedding_matrix : word2vec词向量矩阵
        is_embedding_training : embedding层是否加入训练
        embedding_dropout_rate : embedding层dropout的比率
    """

    input_list =[ 
        tf.keras.layers.Input(shape=(sentence_maxlen,),name="input{}".format(i))
        for i in range(input_number)
    ]
    #embedding层
    if not (embedding_matrix is None):
        embedding_layer = tf.keras.layers.Embedding(
            input_dim = vocab_size,
            output_dim = embedding_dim,
            embeddings_initializer = tf.keras.initializers.Constant(embedding_matrix),
            trainable = is_embedding_training,
            input_length = sentence_maxlen,
        )
    else:
        embedding_layer = tf.keras.layers.Embedding(
            input_dim = vocab_size,
            output_dim = embedding_dim,
            trainable = is_embedding_training,
            input_length = sentence_maxlen,
        )
    output_list = [
        embedding_layer(input_list[i]) for i in range(input_number)
    ]

    embedding_dropout_layer = tf.keras.layers.Dropout(embedding_dropout_rate)
    cnn_layer0 = tf.keras.layers.Conv1D(filters=128, kernel_size=10, strides=1, padding='valid',activation="tanh")
    cnn_layer1 = tf.keras.layers.MaxPool1D(2, padding='valid')
    cnn_layer2 = tf.keras.layers.Flatten()
    for layer in [embedding_dropout_layer,cnn_layer0,cnn_layer1,cnn_layer2]:
        output_list = [
            layer(output_list[i]) for i in range(input_number)
        ]
    output = tf.keras.layers.Concatenate()(output_list)
    
    # output = tf.keras.layers.Concatenate()(output_list)
    # output = tf.keras.layers.Dropout(embedding_dropout_rate)(output)
    # output = tf.keras.layers.Conv1D(filters=512, kernel_size=20, strides=1, padding='valid',activation="tanh")(output)
    # output = tf.keras.layers.MaxPool1D(2, padding='valid')(output)
    # output = tf.keras.layers.Flatten()(output)

    output = tf.keras.layers.Dropout(0.5)(output)
    #output = tf.keras.layers.Dense(16, activation='tanh',kernel_regularizer=tf.keras.regularizers.l2())(output)

    if tag_size > 2:
        output = tf.keras.layers.Dense(tag_size, activation='softmax',use_bias=True,kernel_regularizer=tf.keras.regularizers.l2())(output)
        print("这是一个多分类模型")
    elif tag_size == 2:
        output = tf.keras.layers.Dense(1, activation='sigmoid',use_bias=True,kernel_regularizer=tf.keras.regularizers.l2())(output)
        print("这是一个二分类模型")
    else:
        raise Exception("类别错误")

    model = tf.keras.Model(inputs=input_list, outputs=output, name='multi_textcnn')

    if tag_size > 2:
        metric_list = [
            tf.keras.metrics.SparseCategoricalAccuracy(),
        ]
        model.compile(
            loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
            optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate,
                clipnorm=1.0,
                clipvalue=0.5,
            ),
            metrics = metric_list,
        )
    else:
        metric_list = [
            tf.keras.metrics.BinaryAccuracy(),
            tf.keras.metrics.AUC(num_thresholds=10000)
        ]
        model.compile(
            loss = tf.keras.losses.BinaryCrossentropy(),
            optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3,
                clipnorm=1.0,
                clipvalue=0.5,
            ),
            metrics = metric_list,
        )
    return model


In [66]:
import json
import gensim
import os
import LAC
import tensorflow as tf
import numpy as np
import pandas as pd
import sklearn.utils

APP_DIR  = os.path.dirname(os.path.realpath('__file__'))

def trans_gensim_word2vec2tf_embedding(word2vector_file_path:str):
    """把gensim的word2vec结果转化为tf.keras.layers.Embedding需要的结果
    """

    word2vec_model = gensim.models.Word2Vec.load(word2vector_file_path)

    #所有的词
    word_list = [word for word, word_info in word2vec_model.wv.vocab.items()]

    #词到index的映射
    word2index_dict = {"<PADDING>": 0, "<UNK>":1}

    #保存特殊词的padding
    specical_word_count = len(word2index_dict)

    #词到词向量的映射
    word2vector_dict = {}

    #初始化embeddings_matrix

    embeddings_matrix = np.zeros((len(word_list) + specical_word_count, word2vec_model.vector_size))
    #初始化unk为-1,1分布
    embeddings_matrix[word2index_dict["<UNK>"]] = (1 / np.sqrt(len(word_list) + specical_word_count) * (2 * np.random.rand(word2vec_model.vector_size) - 1))

    for i,word in enumerate(word_list):
        #从0开始
        word_index = i + specical_word_count
        word2index_dict[str(word)] = word_index
        word2vector_dict[str(word)] = word2vec_model.wv[word] # 词语：词向量
        embeddings_matrix[word_index] = word2vec_model.wv[word]  # 词向量矩阵

    #写入文件
    with open(os.path.join(APP_DIR,"data","word2index.json"),"w",encoding="utf8") as f:
        json.dump(word2index_dict,f,ensure_ascii=False)

    return embeddings_matrix,word2vector_dict,word2index_dict


def trans2index(word2index_dict,word):
    """转换"""
    if word in word2index_dict:
        return word2index_dict[word]
    else:
        if "<UNK>" in word2index_dict:
            return word2index_dict["<UNK>"]
        else:
            raise ValueError("没有这个值，请检查")


def trans_data2tf_data(data_file_path:str,x_max_length:int=None,word2index_dict=None):
    """把data文件转化为tf.data
    """

    tag2index_dict = {}
    tag_index_count = len(tag2index_dict)
    lac = LAC.LAC(mode="seg")

    df = pd.read_csv(data_file_path)

    x_list = []
    for doc in df["content"]:
        word_list = lac.run(doc)
        x_list.append([trans2index(word2index_dict,word) for word in word_list])
    x_npa = np.array(x_list)

    y_list = []
    for tag in df["tag"]:
        tag = tag.strip()
        if not (tag in tag2index_dict):
            tag2index_dict[tag] = tag_index_count
            tag_index_count += 1
        y_list.append(tag2index_dict[tag])
    y_npa = np.array(y_list,dtype=np.uint8)

    print("x_list[:1]:{}".format(x_list[:1]))
    print("y_list[:1]:{}".format(y_list[:1]))

    #写入文件
    with open(os.path.join(APP_DIR,"data/tag2index.json"),"w",encoding="utf8") as f:
        json.dump(tag2index_dict,f,ensure_ascii=False)

    if not x_max_length:
        x_max_length0 = np.max(np.array([len(v) for v in x_list]))
        x_max_length = int(np.max(np.percentile(np.array([len(v) for v in x_list]),99.7)))
        print("数据集中最长的句子长度为:{},设定的最长的句子长度为:{}".format(x_max_length0,x_max_length))

    x_npa = tf.keras.preprocessing.sequence.pad_sequences(x_npa,maxlen=x_max_length,dtype=np.int32,truncating="post", padding='post',value=0)

    x_npa,y_npa = sklearn.utils.shuffle(x_npa,y_npa,random_state=0)
    print("x_npa[:1]:{}".format(x_npa[:1]))
    print("y_npa[:1]:{}".format(y_npa[:1]))
    print("x_npa.shape = {}".format(x_npa.shape))
    print("y_npa.shape = {}".format(y_npa.shape))

    return x_npa,y_npa,tag2index_dict



def trans_tokenize_data2tf_data(data_file_path:str,x_max_length:int=None,word2index_dict=None):
    """把已经分好词的data文件转化为tf.data
    """

    tag2index_dict = {}
    tag_index_count = len(tag2index_dict)
    lac = LAC.LAC(mode="seg")

    x_list = []
    y_list = []
    with open(data_file_path) as f:
        for line in f:
            temp_dict = json.loads(line.strip())
            word_list = temp_dict["content_tokenize"]
            tag = temp_dict["tag"].strip()
            if not (tag in tag2index_dict):
                tag2index_dict[tag] = tag_index_count
                tag_index_count += 1
            x_list.append([trans2index(word2index_dict,word) for word in word_list])
            y_list.append(tag2index_dict[tag])
    x_npa = np.array(x_list)
    y_npa = np.array(y_list,dtype=np.uint8)

    print("x_list[:1]:{}".format(x_list[:1]))
    print("y_list[:1]:{}".format(y_list[:1]))

    #写入文件
    with open(os.path.join(APP_DIR,"data/tag2index.json"),"w",encoding="utf8") as f:
        json.dump(tag2index_dict,f,ensure_ascii=False)

    if not x_max_length:
        x_max_length0 = np.max(np.array([len(v) for v in x_list]))
        x_max_length = int(np.max(np.percentile(np.array([len(v) for v in x_list]),99.7)))
        print("数据集中最长的句子长度为:{},设定的最长的句子长度为:{}".format(x_max_length0,x_max_length))

    x_npa = tf.keras.preprocessing.sequence.pad_sequences(x_npa,maxlen=x_max_length,dtype=np.int32,truncating="post", padding='post',value=0)

    x_npa,y_npa = sklearn.utils.shuffle(x_npa,y_npa,random_state=0)
    print("x_npa[:1]:{}".format(x_npa[:1]))
    print("y_npa[:1]:{}".format(y_npa[:1]))
    print("x_npa.shape = {}".format(x_npa.shape))
    print("y_npa.shape = {}".format(y_npa.shape))

    return x_npa,y_npa,tag2index_dict


def trans_multi_input_tokenize_data2npa(data_file_path:str,x_max_length:int=None,word2index_dict=None):
    """把已经分好词的data文件转化为tf.data , 多输入版本
    """

    tag2index_dict = {}
    tag_index_count = len(tag2index_dict)

    x_list = []
    y_list = []
    with open(data_file_path) as f:
        for line in f:
            temp_dict = json.loads(line.strip())
            text_tokenize_list = temp_dict["all_content_tokenize"]
            tag = temp_dict["tag"].strip()
            if not (tag in tag2index_dict):
                tag2index_dict[tag] = tag_index_count
                tag_index_count += 1
            x_list.append([[trans2index(word2index_dict,word) for word in word_list] for word_list in text_tokenize_list])
            y_list.append(tag2index_dict[tag])
    y_npa = np.array(y_list,dtype=np.uint8)

    print("x_list[:1]:{}".format(x_list[:1]))
    print("y_list[:1]:{}".format(y_list[:1]))

    #写入文件
    with open(os.path.join(APP_DIR,"data/tag2index.json"),"w",encoding="utf8") as f:
        json.dump(tag2index_dict,f,ensure_ascii=False)

    if not x_max_length:
        x_max_length0 = np.max(np.array([len(v) for v in x_list]))
        x_max_length = int(np.max(np.percentile(np.array([len(v) for v in x_list]),99.7)))
        print("数据集中最长的句子长度为:{},设定的最长的句子长度为:{}".format(x_max_length0,x_max_length))
    
    for i in range(len(x_list)):
        x_list[i] = tf.keras.preprocessing.sequence.pad_sequences(x_list[i],maxlen=x_max_length,dtype=np.int32,truncating="post", padding='post',value=0)
    x_npa = np.array(x_list,dtype=np.int32)

    x_npa,y_npa = sklearn.utils.shuffle(x_npa,y_npa,random_state=0)
    print("x_npa[:1]:{}".format(x_npa[:1]))
    print("y_npa[:1]:{}".format(y_npa[:1]))
    print("x_npa.shape = {}".format(x_npa.shape))
    print("y_npa.shape = {}".format(y_npa.shape))

    return x_npa,y_npa,tag2index_dict

In [67]:
import os
import time
import collections
import sklearn.model_selection
import gensim
import numpy as np
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')
#from utils import trans_gensim_word2vec2tf_embedding,trans_data2tf_data,trans_tokenize_data2tf_data,trans_multi_input_tokenize_data2npa
#rom model_multi_textcnn import build_model

APP_DIR  = os.path.dirname(os.path.realpath('__file__'))
if not os.path.exists(os.path.join(APP_DIR,"data")):
    os.makedirs(os.path.join(APP_DIR,"data"))

def split_train_eval_test_dataset(dataset):
    """区分训练验证测试集
    """
    dataset_size = tf.data.experimental.cardinality(dataset).numpy()
    print("总共有数据{}条".format(dataset_size))
    dataset = dataset.shuffle(dataset_size,seed=1)
    train_size = int(0.6 * dataset_size)
    eval_size = int(0.2 * dataset_size)
    test_size = int(0.2 * dataset_size)

    train_dataset = dataset.take(train_size)
    test_dataset = dataset.skip(train_size)
    eval_dataset = test_dataset.skip(eval_size)
    test_dataset = test_dataset.take(test_size)
    return train_dataset.prefetch(tf.data.experimental.AUTOTUNE), \
        eval_dataset.prefetch(tf.data.experimental.AUTOTUNE), \
        test_dataset.prefetch(tf.data.experimental.AUTOTUNE)

def split_train_eval_test_npa(x_npa,y_npa):
    x_train,x_test,y_train,y_test =  sklearn.model_selection.train_test_split(x_npa, y_npa, test_size=0.2, random_state=24)
    return x_train,y_train,x_test,y_test

def build_model_callback():
    callback_path = os.path.join(APP_DIR,"model_callback","weights.{epoch:02d}-{val_loss:.2f}.hdf5")
    if not os.path.exists(os.path.dirname(callback_path)):
        os.makedirs(os.path.dirname(callback_path))

    model_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=callback_path,
        monitor='val_loss',
        verbose=1,
        save_best_only=False,
        save_weights_only=True,
        mode='auto',
        save_freq='epoch',
    )

    tf_board_dir = os.path.join(APP_DIR,"model_tensorboard")
    if not os.path.exists(os.path.dirname(tf_board_dir)):
         os.makedirs(os.path.dirname(tf_board_dir))

    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=tf_board_dir,histogram_freq=1,update_freq='batch')
    return model_callback,tensorboard_callback


if __name__ == "__main__":

    input_number = 10
    #句子的最大长度
    sentence_maxlen = 512
    #训练次数
    epochs = 7
    #批大小
    batch_size = 64
    #学习率
    learning_rate = 1e-3
    # learning_rate = tf.keras.optimizers.schedules.ExponentialDecay(
    #     1e-3, decay_steps=100, decay_rate=0.1, staircase=False
    # )

    #embedding layer是否参加训练
    is_embedding_training = False
    # is_embedding_training = True
    #embedding的dropout比率
    #embedding_dropout_rate = 0.35
    embedding_dropout_rate = 0.2


    #数据文件
    #data_csv = os.path.join(APP_DIR,"data","最终数据集.csv")
    #data_jsonl = os.path.join(APP_DIR,"data","最终数据集tokenize.jsonl")
    #data_jsonl = "/mnt1/zhaodachuan/data/predict_user_attribute20200911/raw_data/user_profile/age/最终数据集tokenize.jsonl"
    data_jsonl = r"../data/predict_user_attribute20200911/raw_data/user_profile/trade/updatetexttokenize.jsonl"
    #word2vec路径
    #word2vector_file_path = os.path.join(APP_DIR,"data","word2vector.bin")
    word2vector_file_path = r"../data/dim256/word2vector.bin"
    #模型保存路径
    model_path = os.path.join(APP_DIR,"model_output_trade")
    ###以上是需要修改的部分

    if not os.path.exists(os.path.dirname(model_path)):
        os.makedirs(os.path.dirname(model_path))

    #导入gensim的word2vector
    embedding_matrix,word2vector_dict,word2index_dict = trans_gensim_word2vec2tf_embedding(word2vector_file_path)
    vocab_size,embedding_dim = embedding_matrix.shape

    #划分训练集+验证集，测试集
    #x_npa,y_npa,tag2index_dict = trans_data2tf_data(data_csv,sentence_maxlen,word2index_dict)
    x_npa,y_npa,tag2index_dict = trans_multi_input_tokenize_data2npa(data_jsonl,sentence_maxlen,word2index_dict)
    class_weight_dict = {tag:len(y_npa)/number for tag,number in enumerate(np.bincount(y_npa))}
    x_train_eval,y_train_eval,x_test,y_test = split_train_eval_test_npa(x_npa,y_npa)
    print(x_train_eval[:3])
    print(y_train_eval[:3])
    print(class_weight_dict)
    print(collections.Counter(y_train_eval))
    print(collections.Counter(y_test))
    tag_size = len(tag2index_dict)

x_list[:1]:[[[44577, 1865, 40297, 100263, 15137, 5539, 0, 31305, 21201, 9402, 21556, 7752, 1, 83954, 9802, 1, 28247, 1, 100263, 1643, 30801, 10504, 21753, 100263, 50, 1581, 85970, 45773, 28245, 11267, 2043, 80824, 9837, 1486, 6928, 7066, 10504, 21302, 410, 40017, 2037, 1371, 12457, 40017, 1, 1, 1740, 21530, 1740, 454, 1864, 12625, 2976, 50, 7467, 40017, 891, 1093, 1880, 1873, 44577, 1536, 332, 1, 1740, 3553, 40017, 3722, 332, 58933, 44577, 1, 10191, 1866, 115, 211, 40017, 1, 44577, 8483, 2330, 3790, 44577, 40017, 2037, 1865, 874, 31408, 1866, 115, 7846, 1687, 5181, 50, 1532, 343, 259, 5250, 2894, 2773, 45773, 4396, 343, 7492, 5250, 35457, 45773, 20480, 4801, 41, 44577, 874, 4396, 343, 460, 2894, 1614, 115, 3218, 386, 259, 1, 3246, 259, 16483, 42401, 2845, 4892, 11602, 7727, 259, 4801, 1865, 4801, 263, 10104, 1371, 100263, 45773, 113, 4764, 6938, 1874, 16, 6938, 2142, 1, 3725, 10504, 343, 50, 789, 3880, 1163, 3131, 1, 10504, 77, 332, 1, 1866, 35522, 15268, 17960, 6938, 8879, 1, 1, 19905

In [24]:
x_npa[0][9]

array([ 4357,     1,  1765,     1,  4467, 24895,     0, 31305, 21201,
           1,   750,  2364,  7752,     1,  9802,     1,   104,   439,
           1,  1236,    90, 35097, 31407,  1104,  8444, 11880, 25197,
           1,    78,   521,   458,  3203,  4799,  1357,   458,  4637,
       47022, 62601,  4637,  5223,     1, 14488,  6736,  9566,  7650,
        4467, 24895,  5532,  1352,  1967,  4892,  8694,  2979, 15449,
       77119,     1,  2229,  3216,  4167, 29480,   413,  2420,  5732,
       10940, 11421,  1032,     1,     1,  5912,  2730,  7805,  7650,
           1,     1, 53578,  7807, 17310, 37727,     9, 16646,   387,
        8167, 13474, 54213,  1754,  7910,  3572, 74529,  5912,  2730,
        7805,  3484, 61384,     1,  4637,  3160, 13297,  7826, 14165,
       10945, 11421,   278,  3326, 42484,   827,  3016,   678,  2393,
        2003,  1770,   458,  6932,  1440,   642,   644,     1, 14282,
        1032,  5666,  1405,  1974,  4467, 24895,     1, 18677,  1032,
        4152,  4310,

In [68]:
x_npa1 = x_npa.reshape(6951,5120)

In [69]:
x_npa1 = pd.DataFrame(x_npa1)

In [70]:
y_npa1 = pd.DataFrame(y_npa)

In [71]:
y_npa1.columns = ['trade']

In [72]:
df_data = pd.merge(y_npa1,x_npa1,on = y_npa1.index)

In [73]:
df_data = df_data.drop('key_0',axis = 1)
df_data

,trade,0,1,2,3,4,5,6,7,8,...,5110,5111,5112,5113,5114,5115,5116,5117,5118,5119
0,0,4218,19901,29015,9975,36996,3292,0,3005,21140,...,0,0,0,0,0,0,0,0,0,0
1,4,1,2229,7807,796,0,31305,10361,5558,875,...,0,0,0,0,0,0,0,0,0,0
2,4,4738,13789,0,770,5337,4738,1330,28126,3192,...,0,0,0,0,0,0,0,0,0,0
3,4,5394,2085,0,27113,908,18090,63532,2333,1,...,0,0,0,0,0,0,0,0,0,0
4,1,1,15973,1,61543,0,31305,21201,1,8935,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6946,4,1,21794,0,31305,21201,1,7752,1,9802,...,0,0,0,0,0,0,0,0,0,0
6947,0,5342,407,77351,0,5342,407,6645,39952,14786,...,0,0,0,0,0,0,0,0,0,0
6948,2,9310,21856,14417,1,0,31305,21201,41546,7752,...,0,0,0,0,0,0,0,0,0,0
6949,1,1307,4364,7492,1653,0,31305,10361,7752,1,...,0,0,0,0,0,0,0,0,0,0


In [75]:
df_data.to_csv(r'../data/predict_user_attribute20200911/raw_data/user_profile/trade/trade_dealstopword.csv')

In [56]:
df_data.groupby('trade').count()

,0,1,2,3,4,5,6,7,8,9,...,5110,5111,5112,5113,5114,5115,5116,5117,5118,5119
trade,,,,,,,,,,,,,,,,,,,,,
0,933,933,933,933,933,933,933,933,933,933,...,933,933,933,933,933,933,933,933,933,933
1,2036,2036,2036,2036,2036,2036,2036,2036,2036,2036,...,2036,2036,2036,2036,2036,2036,2036,2036,2036,2036
2,644,644,644,644,644,644,644,644,644,644,...,644,644,644,644,644,644,644,644,644,644
3,1143,1143,1143,1143,1143,1143,1143,1143,1143,1143,...,1143,1143,1143,1143,1143,1143,1143,1143,1143,1143
4,1382,1382,1382,1382,1382,1382,1382,1382,1382,1382,...,1382,1382,1382,1382,1382,1382,1382,1382,1382,1382
5,163,163,163,163,163,163,163,163,163,163,...,163,163,163,163,163,163,163,163,163,163
6,164,164,164,164,164,164,164,164,164,164,...,164,164,164,164,164,164,164,164,164,164
7,98,98,98,98,98,98,98,98,98,98,...,98,98,98,98,98,98,98,98,98,98
8,92,92,92,92,92,92,92,92,92,92,...,92,92,92,92,92,92,92,92,92,92


In [2]:
df_trade = pd.read_csv(r'../data/predict_user_attribute20200911/raw_data/user_profile/trade/trade_dealstopword.csv')

In [3]:
df_trade = df_trade.drop('Unnamed: 0',axis = 1)

In [8]:
df_trade

,trade,0,1,2,3,4,5,6,7,8,...,5110,5111,5112,5113,5114,5115,5116,5117,5118,5119
0,0,4218,19901,29015,9975,36996,3292,0,3005,21140,...,0,0,0,0,0,0,0,0,0,0
1,4,1,2229,7807,796,0,31305,10361,5558,875,...,0,0,0,0,0,0,0,0,0,0
2,4,4738,13789,0,770,5337,4738,1330,28126,3192,...,0,0,0,0,0,0,0,0,0,0
3,4,5394,2085,0,27113,908,18090,63532,2333,1,...,0,0,0,0,0,0,0,0,0,0
4,1,1,15973,1,61543,0,31305,21201,1,8935,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6946,4,1,21794,0,31305,21201,1,7752,1,9802,...,0,0,0,0,0,0,0,0,0,0
6947,0,5342,407,77351,0,5342,407,6645,39952,14786,...,0,0,0,0,0,0,0,0,0,0
6948,2,9310,21856,14417,1,0,31305,21201,41546,7752,...,0,0,0,0,0,0,0,0,0,0
6949,1,1307,4364,7492,1653,0,31305,10361,7752,1,...,0,0,0,0,0,0,0,0,0,0


In [4]:
x_data = df_trade.drop('trade',axis = 1)
y_data = df_trade['trade']

In [5]:
ss = StandardScaler()
x_ss_data = ss.fit_transform(x_data)

In [6]:
csvx_ss_data = pd.DataFrame(x_ss_data)

In [16]:
trade_y_x = pd.merge(y_data,csvx_ss_data,on = y_data.index)
trade_y_x = trade_y_x.drop('key_0',axis = 1)
trade_y_x

,trade,0,1,2,3,4,5,6,7,8,...,5110,5111,5112,5113,5114,5115,5116,5117,5118,5119
0,0,-0.529219,0.185144,0.644530,-0.156265,1.105746,-0.579413,-0.801194,-0.532509,0.500626,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4,-0.704947,-0.584180,-0.307423,-0.649944,-0.675208,0.977356,-0.195338,-0.405795,-0.619559,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4,-0.507550,-0.080933,-0.657852,-0.651342,-0.418290,-0.499055,-0.723423,0.714329,-0.491482,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,-0.480213,-0.590449,-0.657852,0.765476,-0.631498,0.242957,2.913819,-0.565862,-0.667870,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,-0.704947,0.014144,-0.657807,2.617242,-0.675208,0.977356,0.438527,-0.681607,-0.174028,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6946,4,-0.704947,0.267553,-0.657852,0.990937,0.345389,-0.762305,-0.347899,-0.681607,-0.126103,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6947,0,-0.482380,-0.663498,2.814164,-0.692755,-0.418049,-0.739742,-0.412630,1.301291,0.149397,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6948,2,-0.317028,0.270252,-0.010723,-0.692702,-0.675208,0.977356,0.438527,1.380407,-0.239420,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6949,1,-0.650524,-0.491236,-0.321562,-0.603851,-0.675208,0.977356,-0.195338,-0.296899,-0.667870,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
trade_y_x.columns = ['trade',*["c{}".format(v) for v in range(5120)]]

In [25]:
temp_0 = trade_y_x.groupby(trade_y_x.index).filter(lambda x : float(x['trade'])==0)
temp_1 = trade_y_x.groupby(trade_y_x.index).filter(lambda x : float(x['trade'])==1)
temp_2 = trade_y_x.groupby(trade_y_x.index).filter(lambda x : float(x['trade'])==2)
temp_3 = trade_y_x.groupby(trade_y_x.index).filter(lambda x : float(x['trade'])==3)
temp_4 = trade_y_x.groupby(trade_y_x.index).filter(lambda x : float(x['trade'])==4)
temp_5 = trade_y_x.groupby(trade_y_x.index).filter(lambda x : float(x['trade'])==5)
temp_6 = trade_y_x.groupby(trade_y_x.index).filter(lambda x : float(x['trade'])==6)
temp_7 = trade_y_x.groupby(trade_y_x.index).filter(lambda x : float(x['trade'])==7)
temp_8 = trade_y_x.groupby(trade_y_x.index).filter(lambda x : float(x['trade'])==8)
temp_9 = trade_y_x.groupby(trade_y_x.index).filter(lambda x : float(x['trade'])==9)
temp_10 = trade_y_x.groupby(trade_y_x.index).filter(lambda x : float(x['trade'])==10)

In [26]:
temp_0 = temp_0.drop('trade',axis = 1)
temp_1 = temp_1.drop('trade',axis = 1)
temp_2 = temp_2.drop('trade',axis = 1)
temp_3 = temp_3.drop('trade',axis = 1)
temp_4 = temp_4.drop('trade',axis = 1)
temp_5 = temp_5.drop('trade',axis = 1)
temp_6 = temp_6.drop('trade',axis = 1)
temp_7 = temp_7.drop('trade',axis = 1)
temp_8 = temp_8.drop('trade',axis = 1)
temp_9 = temp_9.drop('trade',axis = 1)
temp_10 = temp_10.drop('trade',axis = 1)

In [48]:
#聚类
model = KMeans(n_clusters=2)
model.fit(temp_0)
labels = model.predict(temp_0)
labels = pd.DataFrame(labels)
labels.columns = ['b']
temp_00 = pd.merge(labels,temp_0,on = labels.index)
temp_00 = temp_00.drop('key_0',axis=1)
temp_00.groupby('b').count()

KMeans(n_clusters=2)

,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9,...,c5110,c5111,c5112,c5113,c5114,c5115,c5116,c5117,c5118,c5119
b,,,,,,,,,,,,,,,,,,,,,
0,27,27,27,27,27,27,27,27,27,27,...,27,27,27,27,27,27,27,27,27,27
1,906,906,906,906,906,906,906,906,906,906,...,906,906,906,906,906,906,906,906,906,906


In [49]:
temp_000 = trade_y_x.groupby(trade_y_x.index).filter(lambda x : float(x['trade'])==0)
temp_000 = temp_000['trade']
temp_end_0 = pd.merge(temp_000,temp_00,on = temp_00.index)
temp_end_0 = temp_end_0.drop('key_0',axis = 1)
temp_end_0 = temp_end_0.groupby(temp_end_0.index).filter(lambda x:float(x['b'])==1)
temp_end_0

,trade,b,c0,c1,c2,c3,c4,c5,c6,c7,...,c5110,c5111,c5112,c5113,c5114,c5115,c5116,c5117,c5118,c5119
0,0,1,-0.529219,0.185144,0.644530,-0.156265,1.105746,-0.579413,-0.801194,-0.532509,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,1,-0.581475,-0.681173,-0.657852,-0.308580,0.632346,0.029890,-0.228026,-0.681607,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,1,0.370049,2.130481,-0.598602,2.912190,-0.675208,-0.034519,0.787095,-0.638426,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,1,-0.692654,-0.668287,2.583806,-0.692755,0.684577,0.744394,0.256613,2.471644,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,1,-0.704947,-0.681217,-0.069974,0.768112,-0.675160,-0.217633,-0.801136,-0.681607,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
928,0,1,-0.199932,0.370030,1.338969,-0.261304,0.384911,5.401642,-0.801194,0.872113,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
929,0,1,-0.704988,0.501240,-0.657807,-0.649729,-0.420119,-0.064362,-0.793827,-0.596436,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
930,0,1,2.348563,0.734928,0.089867,-0.692702,-0.675208,0.977356,0.438527,-0.681607,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
931,0,1,-0.026204,-0.681173,-0.657807,-0.202142,-0.530550,-0.079644,-0.801136,-0.681656,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [58]:
#聚类
model = KMeans(n_clusters=2)
model.fit(temp_1)
labels = model.predict(temp_1)
labels = pd.DataFrame(labels)
labels.columns = ['b']
temp_11 = pd.merge(labels,temp_1,on = labels.index)
temp_11 = temp_11.drop('key_0',axis=1)
temp_11.groupby('b').count()

KMeans(n_clusters=2)

,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9,...,c5110,c5111,c5112,c5113,c5114,c5115,c5116,c5117,c5118,c5119
b,,,,,,,,,,,,,,,,,,,,,
0,1844,1844,1844,1844,1844,1844,1844,1844,1844,1844,...,1844,1844,1844,1844,1844,1844,1844,1844,1844,1844
1,192,192,192,192,192,192,192,192,192,192,...,192,192,192,192,192,192,192,192,192,192


In [59]:
temp_111 = trade_y_x.groupby(trade_y_x.index).filter(lambda x : float(x['trade'])==1)
temp_111 = temp_111['trade']
temp_end_1 = pd.merge(temp_111,temp_11,on = temp_11.index)
temp_end_1 = temp_end_1.drop('key_0',axis = 1)
temp_end_1 = temp_end_1.groupby(temp_end_1.index).filter(lambda x:float(x['b'])==0)
temp_end_1

,trade,b,c0,c1,c2,c3,c4,c5,c6,c7,...,c5110,c5111,c5112,c5113,c5114,c5115,c5116,c5117,c5118,c5119
0,1,0,-0.704947,0.014144,-0.657807,2.617242,-0.675208,0.977356,0.438527,-0.681607,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0,2.238842,-0.681173,-0.611888,-0.692755,-0.675160,-0.762305,-0.801136,2.641141,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,0,0.833518,-0.062083,-0.557306,0.681574,-0.675208,-0.527619,-0.423799,-0.470417,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,0,-0.702905,2.019253,-0.138201,-0.677158,-0.675208,0.977356,0.438527,1.967618,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,0,-0.681361,-0.606948,-0.627599,-0.502362,-0.347285,-0.762360,1.029355,0.370618,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2030,1,0,-0.637481,0.451656,1.035850,-0.692755,0.831786,0.415845,3.676972,3.905449,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2032,1,0,0.948447,0.357014,3.305851,-0.692702,-0.675208,-0.557796,0.438527,-0.681607,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2033,1,0,-0.649066,-0.620661,0.164065,0.079145,-0.675208,-0.217633,-0.801136,0.820643,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2034,1,0,-0.662650,-0.310181,-0.657807,-0.402916,-0.674727,-0.762360,0.787095,-0.681607,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [80]:
#聚类
model = KMeans(n_clusters=2)
model.fit(temp_2)
labels = model.predict(temp_2)
labels = pd.DataFrame(labels)
labels.columns = ['b']
temp_22 = pd.merge(labels,temp_2,on = labels.index)
temp_22 = temp_22.drop('key_0',axis=1)
temp_22.groupby('b').count()

KMeans(n_clusters=2)

,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9,...,c5110,c5111,c5112,c5113,c5114,c5115,c5116,c5117,c5118,c5119
b,,,,,,,,,,,,,,,,,,,,,
0,61,61,61,61,61,61,61,61,61,61,...,61,61,61,61,61,61,61,61,61,61
1,583,583,583,583,583,583,583,583,583,583,...,583,583,583,583,583,583,583,583,583,583


In [81]:
temp_222 = trade_y_x.groupby(trade_y_x.index).filter(lambda x : float(x['trade'])==2)
temp_222 = temp_222['trade']
temp_end_2 = pd.merge(temp_222,temp_22,on = temp_22.index)
temp_end_2 = temp_end_2.drop('key_0',axis = 1)
temp_end_2 = temp_end_2.groupby(temp_end_2.index).filter(lambda x:float(x['b'])==1)
temp_end_2

,trade,b,c0,c1,c2,c3,c4,c5,c6,c7,...,c5110,c5111,c5112,c5113,c5114,c5115,c5116,c5117,c5118,c5119
0,2,1,-0.704947,-0.681173,-0.639494,-0.692755,-0.675160,-0.722570,-0.759327,0.923632,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,1,-0.439709,0.364545,1.973574,-0.692755,-0.203349,-0.762305,0.787095,-0.643240,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,1,-0.704947,-0.681217,-0.308994,0.990937,0.345389,1.581047,4.052029,-0.296899,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2,1,-0.488047,-0.681217,-0.611978,-0.692702,1.006338,-0.645101,-0.672550,0.383870,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2,1,-0.642356,1.647651,-0.164594,-0.101675,-0.675208,-0.762305,0.850540,4.252882,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
639,2,1,-0.038289,-0.483052,-0.647394,-0.052302,-0.675208,0.977356,0.438527,-0.514045,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
640,2,1,-0.255730,-0.211882,-0.016828,-0.522638,-0.675208,0.977356,0.438527,-0.681607,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
641,2,1,-0.480422,-0.621140,-0.624187,-0.670543,-0.675208,0.977356,0.438527,-0.681607,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
642,2,1,-0.616062,-0.460284,1.942243,-0.614231,-0.675208,-0.217633,-0.801136,0.720334,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [90]:
#聚类
model = KMeans(n_clusters=2)
model.fit(temp_3)
labels = model.predict(temp_3)
labels = pd.DataFrame(labels)
labels.columns = ['b']
temp_33 = pd.merge(labels,temp_3,on = labels.index)
temp_33 = temp_33.drop('key_0',axis=1)
temp_33.groupby('b').count()

KMeans(n_clusters=2)

,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9,...,c5110,c5111,c5112,c5113,c5114,c5115,c5116,c5117,c5118,c5119
b,,,,,,,,,,,,,,,,,,,,,
0,737,737,737,737,737,737,737,737,737,737,...,737,737,737,737,737,737,737,737,737,737
1,406,406,406,406,406,406,406,406,406,406,...,406,406,406,406,406,406,406,406,406,406


In [91]:
temp_333 = trade_y_x.groupby(trade_y_x.index).filter(lambda x : float(x['trade'])==3)
temp_333 = temp_333['trade']
temp_end_3 = pd.merge(temp_333,temp_33,on = temp_33.index)
temp_end_3 = temp_end_3.drop('key_0',axis = 1)
temp_end_3 = temp_end_3.groupby(temp_end_3.index).filter(lambda x:float(x['b'])==0)
temp_end_3

,trade,b,c0,c1,c2,c3,c4,c5,c6,c7,...,c5110,c5111,c5112,c5113,c5114,c5115,c5116,c5117,c5118,c5119
0,3,0,-0.312861,-0.391980,-0.289334,-0.458743,-0.675208,-0.631263,-0.801136,-0.681607,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,0,-0.691404,-0.666981,-0.594876,-0.572818,-0.518660,-0.576079,-0.777044,-0.447735,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0,-0.581475,-0.681173,-0.657852,-0.308580,0.632346,0.029890,-0.228026,-0.681607,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3,0,0.688251,0.321795,-0.657807,-0.312721,-0.037317,0.580175,-0.801194,0.872113,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,3,0,-0.704947,0.027248,1.010445,-0.031971,-0.675208,0.977356,0.438527,1.663962,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1136,3,0,-0.442542,0.105216,-0.331976,0.501937,-0.636216,-0.762360,1.029355,0.370618,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1138,3,0,-0.704947,-0.627409,-0.419999,-0.478105,0.376438,-0.694728,-0.694420,-0.411652,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1139,3,0,-0.704947,-0.446963,-0.289065,-0.528339,-0.675160,0.906611,-0.801194,-0.644680,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1140,3,0,-0.498090,-0.582396,-0.523148,-0.620901,-0.675208,0.977356,-0.195338,-0.575243,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [111]:
#聚类
model = KMeans(n_clusters=2)
model.fit(temp_4)
labels = model.predict(temp_4)
labels = pd.DataFrame(labels)
labels.columns = ['b']
temp_44 = pd.merge(labels,temp_4,on = labels.index)
temp_44 = temp_44.drop('key_0',axis=1)
temp_44.groupby('b').count()

KMeans(n_clusters=2)

,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9,...,c5110,c5111,c5112,c5113,c5114,c5115,c5116,c5117,c5118,c5119
b,,,,,,,,,,,,,,,,,,,,,
0,1365,1365,1365,1365,1365,1365,1365,1365,1365,1365,...,1365,1365,1365,1365,1365,1365,1365,1365,1365,1365
1,17,17,17,17,17,17,17,17,17,17,...,17,17,17,17,17,17,17,17,17,17


In [112]:
temp_444 = trade_y_x.groupby(trade_y_x.index).filter(lambda x : float(x['trade'])==4)
temp_444 = temp_444['trade']
temp_end_4 = pd.merge(temp_444,temp_44,on = temp_44.index)
temp_end_4 = temp_end_4.drop('key_0',axis = 1)
temp_end_4 = temp_end_4.groupby(temp_end_4.index).filter(lambda x:float(x['b'])==0)
temp_end_4

,trade,b,c0,c1,c2,c3,c4,c5,c6,c7,...,c5110,c5111,c5112,c5113,c5114,c5115,c5116,c5117,c5118,c5119
0,4,0,-0.704947,-0.584180,-0.307423,-0.649944,-0.675208,0.977356,-0.195338,-0.405795,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4,0,-0.507550,-0.080933,-0.657852,-0.651342,-0.418290,-0.499055,-0.723423,0.714329,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4,0,-0.480213,-0.590449,-0.657852,0.765476,-0.631498,0.242957,2.913819,-0.565862,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,0,-0.704947,-0.681173,0.561400,-0.692755,-0.331351,-0.762305,-0.347899,-0.681607,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0,1.374037,-0.491541,1.581580,-0.692755,-0.465466,-0.400079,-0.750263,-0.642496,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1377,4,0,-0.700155,-0.549441,3.535984,3.049607,-0.431432,-0.687059,-0.801194,0.872113,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1378,4,0,-0.704947,-0.681173,-0.657852,-0.692702,1.534761,-0.291156,-0.801136,-0.265233,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1379,4,0,1.165847,-0.681173,2.669854,-0.692755,0.831786,-0.217633,1.241852,-0.387331,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1380,4,0,-0.255730,-0.211882,-0.016828,-0.522638,-0.675208,0.977356,0.438527,-0.681607,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [113]:
#聚类
model = KMeans(n_clusters=2)
model.fit(temp_5)
labels = model.predict(temp_5)
labels = pd.DataFrame(labels)
labels.columns = ['b']
temp_55 = pd.merge(labels,temp_5,on = labels.index)
temp_55 = temp_55.drop('key_0',axis=1)
temp_55.groupby('b').count()

KMeans(n_clusters=2)

,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9,...,c5110,c5111,c5112,c5113,c5114,c5115,c5116,c5117,c5118,c5119
b,,,,,,,,,,,,,,,,,,,,,
0,161,161,161,161,161,161,161,161,161,161,...,161,161,161,161,161,161,161,161,161,161
1,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2


In [114]:
temp_555 = trade_y_x.groupby(trade_y_x.index).filter(lambda x : float(x['trade'])==5)
temp_555 = temp_555['trade']
temp_end_5 = pd.merge(temp_555,temp_55,on = temp_55.index)
temp_end_5 = temp_end_5.drop('key_0',axis = 1)
temp_end_5 = temp_end_5.groupby(temp_end_5.index).filter(lambda x:float(x['b'])==0)
temp_end_5

,trade,b,c0,c1,c2,c3,c4,c5,c6,c7,...,c5110,c5111,c5112,c5113,c5114,c5115,c5116,c5117,c5118,c5119
0,5,0,-0.668693,0.962826,0.539585,0.391412,-0.579845,-0.633375,-0.801194,-0.681607,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5,0,-0.515217,0.048927,-0.560224,-0.677158,-0.675208,-0.677000,-0.432278,0.950484,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5,0,-0.607269,0.238907,-0.471842,0.221618,-0.675208,0.807413,0.784230,-0.636589,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5,0,0.410762,0.484436,-0.511028,-0.368656,-0.370151,-0.732906,-0.578757,0.252243,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,0,-0.468129,-0.544870,-0.136630,-0.387050,-0.012911,-0.762360,1.029355,0.370618,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158,5,0,-0.704947,-0.681217,-0.069974,0.768112,-0.675160,-0.217633,-0.801136,-0.681607,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
159,5,0,0.770344,-0.663977,-0.657807,-0.692755,-0.061098,-0.762305,-0.574196,-0.277096,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
160,5,0,-0.704947,-0.434904,-0.657852,0.990937,0.345389,-0.331557,-0.801136,0.720334,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
161,5,0,-0.652399,0.501284,-0.600846,-0.436746,-0.675160,-0.608812,0.787153,-0.055037,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [115]:
#聚类
model = KMeans(n_clusters=2)
model.fit(temp_6)
labels = model.predict(temp_6)
labels = pd.DataFrame(labels)
labels.columns = ['b']
temp_66 = pd.merge(labels,temp_6,on = labels.index)
temp_66 = temp_66.drop('key_0',axis=1)
temp_66.groupby('b').count()

KMeans(n_clusters=2)

,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9,...,c5110,c5111,c5112,c5113,c5114,c5115,c5116,c5117,c5118,c5119
b,,,,,,,,,,,,,,,,,,,,,
0,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
1,162,162,162,162,162,162,162,162,162,162,...,162,162,162,162,162,162,162,162,162,162


In [116]:
temp_666 = trade_y_x.groupby(trade_y_x.index).filter(lambda x : float(x['trade'])==6)
temp_666 = temp_666['trade']
temp_end_6 = pd.merge(temp_666,temp_66,on = temp_66.index)
temp_end_6 = temp_end_6.drop('key_0',axis = 1)
temp_end_6 = temp_end_6.groupby(temp_end_6.index).filter(lambda x:float(x['b'])==1)
temp_end_6

,trade,b,c0,c1,c2,c3,c4,c5,c6,c7,...,c5110,c5111,c5112,c5113,c5114,c5115,c5116,c5117,c5118,c5119
0,6,1,-0.468129,-0.544870,-0.136630,-0.387050,-0.012911,-0.762360,1.029355,0.370618,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,6,1,2.361731,-0.681173,-0.552010,-0.394849,-0.668710,-0.572467,-0.801194,0.872113,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,6,1,2.054488,-0.072792,1.383227,-0.692755,-0.498008,-0.365401,-0.675825,-0.681607,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,6,1,1.079504,-0.679040,-0.657807,-0.692755,0.831786,0.415845,0.694763,-0.643240,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,6,1,-0.704947,0.573941,-0.613010,1.231456,-0.442215,0.472863,-0.754707,-0.681607,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,6,1,-0.352616,0.068779,-0.305538,-0.692702,-0.675208,-0.365401,-0.801136,-0.296899,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
160,6,1,0.006717,-0.681217,0.747320,0.447508,-0.675160,-0.331557,-0.801136,-0.195151,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
161,6,1,-0.704947,4.080216,1.959345,4.781650,-0.675160,-0.762360,1.029355,0.370618,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
162,6,1,-0.162719,-0.066306,-0.034513,-0.692702,-0.675160,-0.762360,-0.383510,-0.676197,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [117]:
#聚类
model = KMeans(n_clusters=2)
model.fit(temp_7)
labels = model.predict(temp_7)
labels = pd.DataFrame(labels)
labels.columns = ['b']
temp_77 = pd.merge(labels,temp_7,on = labels.index)
temp_77 = temp_77.drop('key_0',axis=1)
temp_77.groupby('b').count()

KMeans(n_clusters=2)

,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9,...,c5110,c5111,c5112,c5113,c5114,c5115,c5116,c5117,c5118,c5119
b,,,,,,,,,,,,,,,,,,,,,
0,5,5,5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
1,93,93,93,93,93,93,93,93,93,93,...,93,93,93,93,93,93,93,93,93,93


In [118]:
temp_777 = trade_y_x.groupby(trade_y_x.index).filter(lambda x : float(x['trade'])==7)
temp_777 = temp_777['trade']
temp_end_7 = pd.merge(temp_777,temp_77,on = temp_77.index)
temp_end_7 = temp_end_7.drop('key_0',axis = 1)
temp_end_7 = temp_end_7.groupby(temp_end_7.index).filter(lambda x:float(x['b'])==1)
temp_end_7

,trade,b,c0,c1,c2,c3,c4,c5,c6,c7,...,c5110,c5111,c5112,c5113,c5114,c5115,c5116,c5117,c5118,c5119
0,7,1,2.070740,-0.681173,-0.657852,0.990937,0.345389,-0.462877,-0.793827,-0.564323,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,7,1,-0.418873,-0.681173,4.371815,-0.393504,-0.675208,0.977356,0.438527,-0.681607,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,7,1,0.905568,-0.681173,-0.657807,1.336657,1.982647,-0.762360,1.029355,0.370618,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,7,1,0.410762,0.484436,-0.511028,-0.368656,-0.370151,-0.732906,-0.578757,0.252243,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,7,1,-0.691695,-0.681173,1.576822,0.274648,4.181112,-0.762360,1.029355,-0.681607,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,7,1,-0.532011,-0.240135,-0.463987,-0.692755,0.831786,0.415845,-0.347899,-0.681607,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
94,7,1,-0.645357,1.008275,-0.218458,-0.565934,-0.675208,-0.640988,-0.049502,-0.434532,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
95,7,1,-0.043622,-0.681173,-0.080432,-0.692702,-0.525977,-0.762360,1.029355,0.370618,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96,7,1,-0.164011,-0.681173,-0.657807,2.989692,-0.675208,-0.557796,0.438527,3.139310,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [122]:
#聚类
model = KMeans(n_clusters=2)
model.fit(temp_8)
labels = model.predict(temp_8)
labels = pd.DataFrame(labels)
labels.columns = ['b']
temp_88 = pd.merge(labels,temp_8,on = labels.index)
temp_88 = temp_88.drop('key_0',axis=1)
temp_88.groupby('b').count()

KMeans(n_clusters=2)

,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9,...,c5110,c5111,c5112,c5113,c5114,c5115,c5116,c5117,c5118,c5119
b,,,,,,,,,,,,,,,,,,,,,
0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1,91,91,91,91,91,91,91,91,91,91,...,91,91,91,91,91,91,91,91,91,91


In [130]:
temp_888 = trade_y_x.groupby(trade_y_x.index).filter(lambda x : float(x['trade'])==8)
temp_888 = temp_888['trade']
temp_end_8 = pd.merge(temp_888,temp_88,on = temp_88.index)
temp_end_8 = temp_end_8.drop('key_0',axis = 1)
temp_end_8 = temp_end_8.groupby(temp_end_8.index).filter(lambda x:float(x['b'])==1)
temp_end_8

,trade,b,c0,c1,c2,c3,c4,c5,c6,c7,...,c5110,c5111,c5112,c5113,c5114,c5115,c5116,c5117,c5118,c5119
0,8,1,-0.704947,-0.681173,-0.457344,-0.692702,2.075700,-0.762360,1.029355,0.370618,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,8,1,-0.043622,-0.681173,-0.080432,-0.692702,-0.525977,-0.762360,1.029355,0.370618,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,8,1,-0.262189,-0.653355,-0.520634,-0.692218,-0.675208,0.977356,0.438527,-0.681607,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,8,1,-0.692654,-0.668287,2.583806,-0.692755,0.684577,0.744394,0.256613,2.471644,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,8,1,-0.704947,2.243233,-0.557351,1.419860,0.074751,-0.758414,-0.645769,-0.384850,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,8,1,-0.424749,-0.676733,-0.653184,-0.456431,-0.484722,0.411066,-0.343513,-0.681607,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
88,8,1,-0.581475,-0.681173,-0.657852,-0.308580,0.632346,0.029890,-0.228026,-0.681607,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
89,8,1,-0.704947,0.451656,-0.359357,-0.692702,-0.615708,2.122551,-0.801194,-0.031610,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
90,8,1,-0.704947,-0.114585,0.032996,2.570665,-0.470762,-0.762305,-0.801194,0.872113,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [126]:
#聚类
model = KMeans(n_clusters=2)
model.fit(temp_9)
labels = model.predict(temp_9)
labels = pd.DataFrame(labels)
labels.columns = ['b']
temp_99 = pd.merge(labels,temp_9,on = labels.index)
temp_99 = temp_99.drop('key_0',axis=1)
temp_99.groupby('b').count()

KMeans(n_clusters=2)

,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9,...,c5110,c5111,c5112,c5113,c5114,c5115,c5116,c5117,c5118,c5119
b,,,,,,,,,,,,,,,,,,,,,
0,158,158,158,158,158,158,158,158,158,158,...,158,158,158,158,158,158,158,158,158,158
1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [131]:
temp_999 = trade_y_x.groupby(trade_y_x.index).filter(lambda x : float(x['trade'])==9)
temp_999 = temp_999['trade']
temp_end_9 = pd.merge(temp_999,temp_99,on = temp_99.index)
temp_end_9 = temp_end_9.drop('key_0',axis = 1)
temp_end_9 = temp_end_9.groupby(temp_end_9.index).filter(lambda x:float(x['b'])==0)
temp_end_9

,trade,b,c0,c1,c2,c3,c4,c5,c6,c7,...,c5110,c5111,c5112,c5113,c5114,c5115,c5116,c5117,c5118,c5119
0,9,0,1.018038,2.914389,-0.102561,-0.524306,-0.062590,-0.762305,-0.801194,0.872113,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,9,0,-0.608561,0.451133,-0.657807,3.010345,-0.675208,0.977356,0.438527,0.549894,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,9,0,-0.704947,-0.681173,-0.657807,0.322677,-0.393258,-0.762360,1.029355,0.370618,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,9,0,-0.610686,-0.681173,-0.657852,-0.577390,-0.675160,-0.695061,-0.770437,-0.569336,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,9,0,-0.498090,-0.582396,-0.523148,-0.620901,-0.675208,0.977356,-0.195338,-0.575243,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,9,0,-0.567723,-0.537774,-0.657807,-0.692755,0.831786,0.415845,-0.801136,-0.296899,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
155,9,0,-0.532844,-0.617527,0.694085,-0.620578,-0.491317,-0.692783,-0.352635,0.539719,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
156,9,0,1.177265,1.008275,-0.657852,-0.439274,-0.116699,-0.509502,-0.456135,1.700790,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
157,9,0,-0.284858,-0.087463,0.586626,1.336657,2.388219,-0.676833,-0.801194,0.872113,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [129]:
#聚类
model = KMeans(n_clusters=2)
model.fit(temp_10)
labels = model.predict(temp_10)
labels = pd.DataFrame(labels)
labels.columns = ['b']
temp_ten = pd.merge(labels,temp_10,on = labels.index)
temp_ten = temp_ten.drop('key_0',axis=1)
temp_ten.groupby('b').count()

KMeans(n_clusters=2)

,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9,...,c5110,c5111,c5112,c5113,c5114,c5115,c5116,c5117,c5118,c5119
b,,,,,,,,,,,,,,,,,,,,,
0,135,135,135,135,135,135,135,135,135,135,...,135,135,135,135,135,135,135,135,135,135
1,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2


In [132]:
temp_doubleten = trade_y_x.groupby(trade_y_x.index).filter(lambda x : float(x['trade'])==10)
temp_doubleten = temp_doubleten['trade']
temp_end_10 = pd.merge(temp_doubleten,temp_ten,on = temp_ten.index)
temp_end_10 = temp_end_10.drop('key_0',axis = 1)
temp_end_10 = temp_end_10.groupby(temp_end_10.index).filter(lambda x:float(x['b'])==0)
temp_end_10

,trade,b,c0,c1,c2,c3,c4,c5,c6,c7,...,c5110,c5111,c5112,c5113,c5114,c5115,c5116,c5117,c5118,c5119
0,10,0,-0.088628,-0.681217,-0.442352,-0.218116,-0.423537,-0.325833,0.063705,-0.214608,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10,0,-0.607269,0.238907,-0.471842,0.221618,-0.675208,0.807413,0.784230,-0.636589,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10,0,-0.704947,0.027248,1.010445,-0.031971,-0.675208,0.977356,0.438527,1.663962,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10,0,-0.471421,-0.681217,-0.217875,-0.692702,-0.675160,-0.762305,-0.122771,-0.535735,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10,0,-0.198890,-0.474955,-0.657852,1.929351,1.695738,-0.217633,-0.801136,0.820643,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,10,0,-0.704947,4.080216,1.959345,4.781650,-0.675160,-0.762360,1.029355,0.370618,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
133,10,0,-0.704947,2.920876,1.496924,-0.692755,0.831786,-0.762305,-0.801136,0.820643,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
134,10,0,1.887094,-0.681173,-0.657852,-0.165570,-0.675160,-0.762305,0.968717,-0.681607,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
135,10,0,-0.652399,0.501284,-0.600846,-0.436746,-0.675160,-0.608812,0.787153,-0.055037,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [133]:
x_npa1 = pd.concat([temp_end_0, temp_end_1], axis=0, ignore_index=True)
x_npa1 = pd.concat([x_npa1, temp_end_2], axis=0, ignore_index=True)
x_npa1 = pd.concat([x_npa1, temp_end_3], axis=0, ignore_index=True)
x_npa1 = pd.concat([x_npa1, temp_end_4], axis=0, ignore_index=True)
x_npa1 = pd.concat([x_npa1, temp_end_5], axis=0, ignore_index=True)
x_npa1 = pd.concat([x_npa1, temp_end_6], axis=0, ignore_index=True)
x_npa1 = pd.concat([x_npa1, temp_end_7], axis=0, ignore_index=True)
x_npa1 = pd.concat([x_npa1, temp_end_8], axis=0, ignore_index=True)
x_npa1 = pd.concat([x_npa1, temp_end_9], axis=0, ignore_index=True)
x_npa1 = pd.concat([x_npa1, temp_end_10], axis=0, ignore_index=True)
x_npa1

,trade,b,c0,c1,c2,c3,c4,c5,c6,c7,...,c5110,c5111,c5112,c5113,c5114,c5115,c5116,c5117,c5118,c5119
0,0,1,-0.529219,0.185144,0.644530,-0.156265,1.105746,-0.579413,-0.801194,-0.532509,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,1,-0.581475,-0.681173,-0.657852,-0.308580,0.632346,0.029890,-0.228026,-0.681607,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,1,0.370049,2.130481,-0.598602,2.912190,-0.675208,-0.034519,0.787095,-0.638426,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,1,-0.692654,-0.668287,2.583806,-0.692755,0.684577,0.744394,0.256613,2.471644,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,1,-0.704947,-0.681217,-0.069974,0.768112,-0.675160,-0.217633,-0.801136,-0.681607,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6230,10,0,-0.704947,4.080216,1.959345,4.781650,-0.675160,-0.762360,1.029355,0.370618,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6231,10,0,-0.704947,2.920876,1.496924,-0.692755,0.831786,-0.762305,-0.801136,0.820643,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6232,10,0,1.887094,-0.681173,-0.657852,-0.165570,-0.675160,-0.762305,0.968717,-0.681607,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6233,10,0,-0.652399,0.501284,-0.600846,-0.436746,-0.675160,-0.608812,0.787153,-0.055037,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [134]:
x_npa1 = x_npa1.drop('b',axis = 1)
x_npa1.groupby('trade').count()

,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9,...,c5110,c5111,c5112,c5113,c5114,c5115,c5116,c5117,c5118,c5119
trade,,,,,,,,,,,,,,,,,,,,,
0,906,906,906,906,906,906,906,906,906,906,...,906,906,906,906,906,906,906,906,906,906
1,1844,1844,1844,1844,1844,1844,1844,1844,1844,1844,...,1844,1844,1844,1844,1844,1844,1844,1844,1844,1844
2,583,583,583,583,583,583,583,583,583,583,...,583,583,583,583,583,583,583,583,583,583
3,737,737,737,737,737,737,737,737,737,737,...,737,737,737,737,737,737,737,737,737,737
4,1365,1365,1365,1365,1365,1365,1365,1365,1365,1365,...,1365,1365,1365,1365,1365,1365,1365,1365,1365,1365
5,161,161,161,161,161,161,161,161,161,161,...,161,161,161,161,161,161,161,161,161,161
6,162,162,162,162,162,162,162,162,162,162,...,162,162,162,162,162,162,162,162,162,162
7,93,93,93,93,93,93,93,93,93,93,...,93,93,93,93,93,93,93,93,93,93
8,91,91,91,91,91,91,91,91,91,91,...,91,91,91,91,91,91,91,91,91,91


In [136]:
x_npa1.to_csv('../data/predict_user_attribute20200911/raw_data/user_profile/trade/newdealspdata/trade_new.csv')

In [6]:
x_npa1 = pd.read_csv('../data/predict_user_attribute20200911/raw_data/user_profile/trade/newdealspdata/trade_new.csv')

In [7]:
x_temp = x_npa1.drop('trade',axis = 1)
y_temp = x_npa1['trade']

In [8]:
x_temp = x_temp.drop('Unnamed: 0',axis = 1)
x_temp

,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9,...,c5110,c5111,c5112,c5113,c5114,c5115,c5116,c5117,c5118,c5119
0,-0.529219,0.185144,0.644530,-0.156265,1.105746,-0.579413,-0.801194,-0.532509,0.500626,-0.161731,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.581475,-0.681173,-0.657852,-0.308580,0.632346,0.029890,-0.228026,-0.681607,-0.667870,-0.303234,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.370049,2.130481,-0.598602,2.912190,-0.675208,-0.034519,0.787095,-0.638426,-0.126103,-0.606478,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.692654,-0.668287,2.583806,-0.692755,0.684577,0.744394,0.256613,2.471644,-0.538965,-0.606478,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.704947,-0.681217,-0.069974,0.768112,-0.675160,-0.217633,-0.801136,-0.681607,1.263832,1.331741,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6230,-0.704947,4.080216,1.959345,4.781650,-0.675160,-0.762360,1.029355,0.370618,-0.667870,-0.606478,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6231,-0.704947,2.920876,1.496924,-0.692755,0.831786,-0.762305,-0.801136,0.820643,-0.667870,-0.606478,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6232,1.887094,-0.681173,-0.657852,-0.165570,-0.675160,-0.762305,0.968717,-0.681607,0.893478,-0.606478,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6233,-0.652399,0.501284,-0.600846,-0.436746,-0.675160,-0.608812,0.787153,-0.055037,-0.667926,-0.606478,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [126]:
x_temp0 = x_temp.T[5000:5120]
x_temp0

,0,1,2,3,4,5,6,7,8,9,...,6225,6226,6227,6228,6229,6230,6231,6232,6233,6234
c5000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
c5001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
c5002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
c5003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
c5004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
c5115,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
c5116,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
c5117,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
c5118,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [127]:
x_temp0 = x_temp0.T

In [128]:
#rfe取前1000个特征中最有用的50个
estimator = SVC(kernel = "linear")
print(1)
selector = RFE(estimator,n_features_to_select = 1,step = 1)
print(2)
selector = selector.fit(x_temp0,y_temp)
print(3)
rank_rfe0 = selector.ranking_
print('done')

1
2
3
done


In [129]:
rank_rfe0 = pd.DataFrame(rank_rfe0)

In [130]:
rank_rfe0

,0
0,120
1,119
2,118
3,117
4,116
...,...
115,5
116,4
117,3
118,2


In [25]:
rank_rfe1 = pd.read_csv('../data/predict_user_attribute20200911/raw_data/user_profile/trade/newdealspdata/trade_new1.csv')

In [26]:
rank_rfe1 = rank_rfe1.drop('Unnamed: 0',axis = 1)

In [27]:
rank_rfe1.columns=['c0']
rank_rfe1

,c0
0,1
1,86
2,18
3,61
4,77
...,...
395,3
396,2
397,1
398,8


In [131]:
rank_rfe0.columns = ['c0']
rank_rfe2 = pd.concat([rank_rfe2, rank_rfe0], axis=0, ignore_index=True)

In [58]:
rank_rfe1.to_csv('../data/predict_user_attribute20200911/raw_data/user_profile/trade/newdealspdata/trade_new1.csv')

In [11]:
rank_rfe2 = pd.read_csv('../data/predict_user_attribute20200911/raw_data/user_profile/trade/newdealspdata/trade_new2.csv')
rank_rfe2 = rank_rfe2.drop('Unnamed: 0',axis = 1)
rank_rfe2

,c0
0,1
1,86
2,18
3,61
4,77
...,...
4545,5
4546,4
4547,3
4548,2


In [132]:
rank_rfe2

,c0
0,1
1,86
2,18
3,61
4,77
...,...
5115,5
5116,4
5117,3
5118,2


In [133]:
rank_rfe2.to_csv('../data/predict_user_attribute20200911/raw_data/user_profile/trade/newdealspdata/trade_new2.csv')

In [2]:
rank_rfe3 = pd.read_csv('../data/predict_user_attribute20200911/raw_data/user_profile/trade/newdealspdata/trade_new2.csv')

In [3]:
rank_rfe3.columns = ['index_a','rank']

In [4]:
rank_rfe3.groupby('rank').count()

,index_a
rank,
1,366
2,112
3,112
4,112
5,112
...,...
116,1
117,1
118,1


In [5]:
rank_rfe3 = rank_rfe3.drop('index_a',axis = 1)

In [5]:
temp_rfe_1 = rank_rfe3.groupby(rank_rfe3.index).filter(lambda x : float(x['rank'])==1)

In [6]:
temp_rfe_1.count()

index_a    366
rank       366
dtype: int64

In [7]:
temp_rfe_1 = temp_rfe_1.drop('index_a',axis = 1)
temp_rfe_1

,rank
0,1
10,1
27,1
44,1
59,1
...,...
4890,1
4900,1
4902,1
4925,1


In [9]:
x_npa1 = x_npa1.drop('Unnamed: 0',axis = 1)

In [10]:
x_npa1

,trade,c0,c1,c2,c3,c4,c5,c6,c7,c8,...,c5110,c5111,c5112,c5113,c5114,c5115,c5116,c5117,c5118,c5119
0,0,-0.529219,0.185144,0.644530,-0.156265,1.105746,-0.579413,-0.801194,-0.532509,0.500626,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,-0.581475,-0.681173,-0.657852,-0.308580,0.632346,0.029890,-0.228026,-0.681607,-0.667870,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0.370049,2.130481,-0.598602,2.912190,-0.675208,-0.034519,0.787095,-0.638426,-0.126103,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,-0.692654,-0.668287,2.583806,-0.692755,0.684577,0.744394,0.256613,2.471644,-0.538965,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,-0.704947,-0.681217,-0.069974,0.768112,-0.675160,-0.217633,-0.801136,-0.681607,1.263832,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6230,10,-0.704947,4.080216,1.959345,4.781650,-0.675160,-0.762360,1.029355,0.370618,-0.667870,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6231,10,-0.704947,2.920876,1.496924,-0.692755,0.831786,-0.762305,-0.801136,0.820643,-0.667870,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6232,10,1.887094,-0.681173,-0.657852,-0.165570,-0.675160,-0.762305,0.968717,-0.681607,0.893478,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6233,10,-0.652399,0.501284,-0.600846,-0.436746,-0.675160,-0.608812,0.787153,-0.055037,-0.667926,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
y_data_trade = x_npa1['trade']
x_data_trade = x_npa1.drop('trade',axis = 1)

In [12]:
x_data_trade = x_data_trade.T
x_data_trade

,0,1,2,3,4,5,6,7,8,9,...,6225,6226,6227,6228,6229,6230,6231,6232,6233,6234
c0,-0.529219,-0.581475,0.370049,-0.692654,-0.704947,-0.162719,-0.078751,1.097922,1.787167,-0.414581,...,-0.467920,-0.704947,-0.704947,-0.579391,1.286610,-0.704947,-0.704947,1.887094,-0.652399,-0.700155
c1,0.185144,-0.681173,2.130481,-0.668287,-0.681217,-0.066306,-0.681217,0.877326,0.477993,-0.681217,...,-0.019333,-0.081978,-0.660756,-0.433554,2.265261,4.080216,2.920876,-0.681173,0.501284,-0.549441
c2,0.644530,-0.657852,-0.598602,2.583806,-0.069974,-0.034513,0.747320,-0.657852,0.161910,0.747320,...,-0.562603,-0.424038,-0.639583,0.581464,-0.062522,1.959345,1.496924,-0.657852,-0.600846,3.535984
c3,-0.156265,-0.308580,2.912190,-0.692755,0.768112,-0.692702,-0.165570,0.990937,-0.228335,0.447508,...,0.702603,-0.692702,-0.692755,0.792260,4.829518,4.781650,-0.692755,-0.165570,-0.436746,3.049607
c4,1.105746,0.632346,-0.675208,0.684577,-0.675160,-0.675160,-0.675160,0.345389,-0.675160,1.354721,...,-0.668421,4.668713,0.831786,-0.675208,-0.675208,-0.675160,0.831786,-0.675160,-0.675160,-0.431432
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
c5115,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
c5116,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
c5117,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
c5118,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [13]:
x_oridata = pd.merge(rank_rfe3,x_data_trade,on = rank_rfe3.index)

In [14]:
x_oridata = x_oridata.drop('key_0',axis = 1)

In [28]:
x_oridata = x_oridata.T
x_oridata

,rank,0,1,2,3,4,5,6,7,8,...,6225,6226,6227,6228,6229,6230,6231,6232,6233,6234
0,1.0,-0.529219,-0.581475,0.370049,-0.692654,-0.704947,-0.162719,-0.078751,1.097922,1.787167,...,-0.467920,-0.704947,-0.704947,-0.579391,1.286610,-0.704947,-0.704947,1.887094,-0.652399,-0.700155
1,86.0,0.185144,-0.681173,2.130481,-0.668287,-0.681217,-0.066306,-0.681217,0.877326,0.477993,...,-0.019333,-0.081978,-0.660756,-0.433554,2.265261,4.080216,2.920876,-0.681173,0.501284,-0.549441
2,18.0,0.644530,-0.657852,-0.598602,2.583806,-0.069974,-0.034513,0.747320,-0.657852,0.161910,...,-0.562603,-0.424038,-0.639583,0.581464,-0.062522,1.959345,1.496924,-0.657852,-0.600846,3.535984
3,61.0,-0.156265,-0.308580,2.912190,-0.692755,0.768112,-0.692702,-0.165570,0.990937,-0.228335,...,0.702603,-0.692702,-0.692755,0.792260,4.829518,4.781650,-0.692755,-0.165570,-0.436746,3.049607
4,77.0,1.105746,0.632346,-0.675208,0.684577,-0.675160,-0.675160,-0.675160,0.345389,-0.675160,...,-0.668421,4.668713,0.831786,-0.675208,-0.675208,-0.675160,0.831786,-0.675160,-0.675160,-0.431432
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5115,5.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5116,4.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5117,3.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5118,2.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [15]:
x_oridata = x_oridata.groupby(x_oridata.index).filter(lambda x : float(x['rank'])==1)

In [16]:
x_oridata1 = x_oridata.drop('rank',axis = 1)
x_oridata1

,0,1,2,3,4,5,6,7,8,9,...,6225,6226,6227,6228,6229,6230,6231,6232,6233,6234
0,-0.529219,-0.581475,0.370049,-0.692654,-0.704947,-0.162719,-0.078751,1.097922,1.787167,-0.414581,...,-0.467920,-0.704947,-0.704947,-0.579391,1.286610,-0.704947,-0.704947,1.887094,-0.652399,-0.700155
10,-0.628519,-0.628519,1.675549,0.348649,-0.628519,-0.165314,-0.606826,0.801930,-0.466461,-0.628519,...,0.149539,0.392886,-0.628519,-0.628519,-0.628519,-0.262199,1.602956,-0.628519,-0.031896,-0.397838
27,-0.624254,-0.580238,-0.560022,-0.415155,-0.100426,-0.026362,-0.584603,-0.553681,-0.588876,-0.600592,...,-0.599903,-0.332453,-0.624208,-0.064635,2.872816,-0.117105,-0.624208,-0.110167,-0.504382,-0.515776
44,-0.587999,-0.446588,-0.198879,-0.513954,-0.576883,0.503645,-0.164672,-0.587951,-0.549831,-0.454937,...,-0.357323,-0.518343,0.104792,3.246662,-0.568868,0.394724,-0.587951,1.678924,0.014335,-0.346398
59,-0.606277,1.638715,0.110310,-0.604324,-0.386127,-0.606233,2.128495,1.066129,-0.386970,-0.301457,...,-0.370684,-0.379603,-0.205382,2.052124,-0.562522,-0.606233,-0.572107,-0.502614,-0.268175,-0.581870
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4890,-0.110348,-0.110348,-0.110348,-0.110348,-0.110348,-0.110348,-0.110348,-0.110348,-0.110348,-0.110348,...,-0.110348,-0.110348,-0.110348,-0.110348,-0.110348,-0.110348,-0.110348,-0.110348,-0.110348,-0.110348
4900,-0.096082,-0.096082,-0.096082,-0.096082,-0.096082,-0.096082,-0.096082,-0.096082,-0.096082,-0.096082,...,-0.096082,-0.096082,-0.096082,-0.096082,-0.096082,-0.096082,-0.096082,-0.096082,-0.096082,-0.096082
4902,-0.095219,-0.095219,-0.095219,-0.095219,-0.095219,-0.095219,-0.095219,-0.095219,-0.095219,-0.095219,...,-0.095219,-0.095219,-0.095219,-0.095219,-0.095219,-0.095219,-0.095219,-0.095219,-0.095219,-0.095219
4925,-0.090929,-0.090929,-0.090929,-0.090929,-0.090929,-0.090929,-0.090929,-0.090929,-0.090929,-0.090929,...,-0.090929,-0.090929,-0.090929,-0.090929,-0.090929,-0.090929,-0.090929,-0.090929,-0.090929,-0.090929


In [17]:
x_oridata2 = x_oridata1.T


In [18]:
data_y_x = pd.merge(y_data_trade,x_oridata2,on = y_data_trade.index)
data_y_x 

,key_0,trade,0,10,27,44,59,77,85,88,...,4833,4844,4867,4868,4886,4890,4900,4902,4925,5119
0,0,0,-0.529219,-0.628519,-0.624254,-0.587999,-0.606277,-0.590168,-0.590103,-0.598895,...,1.800348,1.534982,-0.206908,-0.189932,-0.072746,-0.110348,-0.096082,-0.095219,-0.090929,0.0
1,1,0,-0.581475,-0.628519,-0.580238,-0.446588,1.638715,3.719115,-0.459063,-0.577720,...,1.105938,-0.315548,6.086528,-0.051531,-0.072746,-0.110348,-0.096082,-0.095219,-0.090929,0.0
2,2,0,0.370049,1.675549,-0.560022,-0.198879,0.110310,-0.590130,-0.590059,-0.533281,...,-0.354996,-0.328551,-0.206908,-0.189932,-0.072746,-0.110348,-0.096082,-0.095219,-0.090929,0.0
3,3,0,-0.692654,0.348649,-0.415155,-0.513954,-0.604324,-0.520873,-0.561057,-0.555302,...,-0.354996,-0.328551,-0.206908,-0.189932,-0.072746,-0.110348,-0.096082,-0.095219,-0.090929,0.0
4,4,0,-0.704947,-0.628519,-0.100426,-0.576883,-0.386127,-0.464050,-0.570120,1.905627,...,-0.354996,-0.328551,-0.206908,-0.189932,-0.072746,-0.110348,-0.096082,-0.095219,-0.090929,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6230,6230,10,-0.704947,-0.262199,-0.117105,0.394724,-0.606233,-0.590130,0.271915,2.911910,...,-0.098807,-0.294427,-0.206908,-0.189932,-0.072746,-0.110348,-0.096082,-0.095219,-0.090929,0.0
6231,6231,10,-0.704947,1.602956,-0.624208,-0.587951,-0.572107,0.652177,0.264001,-0.553661,...,-0.354996,-0.328551,-0.206908,-0.189932,-0.072746,-0.110348,-0.096082,-0.095219,-0.090929,0.0
6232,6232,10,1.887094,-0.628519,-0.110167,1.678924,-0.502614,-0.458661,2.603447,-0.309895,...,-0.354942,-0.328476,-0.206908,-0.189932,-0.072746,-0.110348,-0.096082,-0.095219,-0.090929,0.0
6233,6233,10,-0.652399,-0.031896,-0.504382,0.014335,-0.268175,-0.501203,-0.075846,0.114062,...,-0.354996,-0.328551,-0.206908,-0.189932,-0.072746,-0.110348,-0.096082,-0.095219,-0.090929,0.0


In [19]:
data_y_x = data_y_x.drop('key_0',axis = 1)
data_y_x

,trade,0,10,27,44,59,77,85,88,89,...,4833,4844,4867,4868,4886,4890,4900,4902,4925,5119
0,0,-0.529219,-0.628519,-0.624254,-0.587999,-0.606277,-0.590168,-0.590103,-0.598895,-0.579149,...,1.800348,1.534982,-0.206908,-0.189932,-0.072746,-0.110348,-0.096082,-0.095219,-0.090929,0.0
1,0,-0.581475,-0.628519,-0.580238,-0.446588,1.638715,3.719115,-0.459063,-0.577720,-0.536888,...,1.105938,-0.315548,6.086528,-0.051531,-0.072746,-0.110348,-0.096082,-0.095219,-0.090929,0.0
2,0,0.370049,1.675549,-0.560022,-0.198879,0.110310,-0.590130,-0.590059,-0.533281,0.211710,...,-0.354996,-0.328551,-0.206908,-0.189932,-0.072746,-0.110348,-0.096082,-0.095219,-0.090929,0.0
3,0,-0.692654,0.348649,-0.415155,-0.513954,-0.604324,-0.520873,-0.561057,-0.555302,-0.511114,...,-0.354996,-0.328551,-0.206908,-0.189932,-0.072746,-0.110348,-0.096082,-0.095219,-0.090929,0.0
4,0,-0.704947,-0.628519,-0.100426,-0.576883,-0.386127,-0.464050,-0.570120,1.905627,-0.359078,...,-0.354996,-0.328551,-0.206908,-0.189932,-0.072746,-0.110348,-0.096082,-0.095219,-0.090929,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6230,10,-0.704947,-0.262199,-0.117105,0.394724,-0.606233,-0.590130,0.271915,2.911910,-0.579097,...,-0.098807,-0.294427,-0.206908,-0.189932,-0.072746,-0.110348,-0.096082,-0.095219,-0.090929,0.0
6231,10,-0.704947,1.602956,-0.624208,-0.587951,-0.572107,0.652177,0.264001,-0.553661,-0.388661,...,-0.354996,-0.328551,-0.206908,-0.189932,-0.072746,-0.110348,-0.096082,-0.095219,-0.090929,0.0
6232,10,1.887094,-0.628519,-0.110167,1.678924,-0.502614,-0.458661,2.603447,-0.309895,-0.579097,...,-0.354942,-0.328476,-0.206908,-0.189932,-0.072746,-0.110348,-0.096082,-0.095219,-0.090929,0.0
6233,10,-0.652399,-0.031896,-0.504382,0.014335,-0.268175,-0.501203,-0.075846,0.114062,-0.068623,...,-0.354996,-0.328551,-0.206908,-0.189932,-0.072746,-0.110348,-0.096082,-0.095219,-0.090929,0.0


In [20]:
data_y_x.columns = ['trade',*["c{}".format(v) for v in range(366)]]

In [22]:
data_y_x.to_csv('../data/predict_user_attribute20200911/raw_data/user_profile/trade/newdealspdata/data_y_x.csv')

In [23]:
x_temp = data_y_x.drop('trade',axis = 1)
y_temp = data_y_x['trade']

In [160]:
x_temp0 = x_temp.T[360:366]
x_temp0

,0,1,2,3,4,5,6,7,8,9,...,6225,6226,6227,6228,6229,6230,6231,6232,6233,6234
c360,-0.072746,-0.072746,-0.072746,-0.072746,-0.072746,-0.072746,-0.072746,-0.072746,-0.072746,-0.072746,...,-0.072746,-0.072746,-0.072746,-0.072746,-0.072746,-0.072746,-0.072746,-0.072746,-0.072746,-0.072746
c361,-0.110348,-0.110348,-0.110348,-0.110348,-0.110348,-0.110348,-0.110348,-0.110348,-0.110348,-0.110348,...,-0.110348,-0.110348,-0.110348,-0.110348,-0.110348,-0.110348,-0.110348,-0.110348,-0.110348,-0.110348
c362,-0.096082,-0.096082,-0.096082,-0.096082,-0.096082,-0.096082,-0.096082,-0.096082,-0.096082,-0.096082,...,-0.096082,-0.096082,-0.096082,-0.096082,-0.096082,-0.096082,-0.096082,-0.096082,-0.096082,-0.096082
c363,-0.095219,-0.095219,-0.095219,-0.095219,-0.095219,-0.095219,-0.095219,-0.095219,-0.095219,-0.095219,...,-0.095219,-0.095219,-0.095219,-0.095219,-0.095219,-0.095219,-0.095219,-0.095219,-0.095219,-0.095219
c364,-0.090929,-0.090929,-0.090929,-0.090929,-0.090929,-0.090929,-0.090929,-0.090929,-0.090929,-0.090929,...,-0.090929,-0.090929,-0.090929,-0.090929,-0.090929,-0.090929,-0.090929,-0.090929,-0.090929,-0.090929
c365,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [161]:
x_temp0 = x_temp0.T

In [162]:
#rfe取前1000个特征中最有用的50个
estimator = SVC(kernel = "linear")
print(1)
selector = RFE(estimator,n_features_to_select = 2,step = 1)
print(2)
selector = selector.fit(x_temp0,y_temp)
print(3)
rank_rfe0 = selector.ranking_
print('done')

1
2
3
done


In [163]:
rank_rfe0 = pd.DataFrame(rank_rfe0)
rank_rfe0

,0
0,1
1,1
2,3
3,2
4,4
5,5


In [48]:
rank_rfe366 = pd.DataFrame(rank_rfe0)
rank_rfe366

,0
0,1
1,3
2,5
3,1
4,1
5,1
6,2
7,1
8,1
9,4


In [164]:
rank_rfe0.columns = ['c0']
#rank_rfe366.columns = ['c0']
rank_rfe366 = pd.concat([rank_rfe366, rank_rfe0], axis=0, ignore_index=True)

In [28]:
rank_rfe366 = pd.read_csv('../data/predict_user_attribute20200911/raw_data/user_profile/trade/newdealspdata/rank_rfe366.csv')
rank_rfe366 = rank_rfe366.drop('Unnamed: 0',axis = 1)
rank_rfe366

,c0
0,1
1,5
2,1
3,2
4,4
...,...
175,6
176,1
177,1
178,1


In [165]:
rank_rfe366

,c0
0,1
1,5
2,1
3,2
4,4
...,...
361,1
362,3
363,2
364,4


In [166]:
rank_rfe366.to_csv('../data/predict_user_attribute20200911/raw_data/user_profile/trade/newdealspdata/rank_rfe366.csv')

In [167]:
rank_rfe366.groupby('c0')['c0'].count()

c0
1    200
2     32
3     32
4     32
5     32
6     21
7      6
8      6
9      5
Name: c0, dtype: int64

In [169]:
rank_rfe366.columns = ['rank']

In [175]:
rank_rfe366
x_temp

,rank
0,1
1,5
2,1
3,2
4,4
...,...
361,1
362,3
363,2
364,4


,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9,...,c356,c357,c358,c359,c360,c361,c362,c363,c364,c365
0,-0.529219,-0.628519,-0.624254,-0.587999,-0.606277,-0.590168,-0.590103,-0.598895,-0.579149,-0.599936,...,1.800348,1.534982,-0.206908,-0.189932,-0.072746,-0.110348,-0.096082,-0.095219,-0.090929,0.0
1,-0.581475,-0.628519,-0.580238,-0.446588,1.638715,3.719115,-0.459063,-0.577720,-0.536888,-0.594151,...,1.105938,-0.315548,6.086528,-0.051531,-0.072746,-0.110348,-0.096082,-0.095219,-0.090929,0.0
2,0.370049,1.675549,-0.560022,-0.198879,0.110310,-0.590130,-0.590059,-0.533281,0.211710,0.012049,...,-0.354996,-0.328551,-0.206908,-0.189932,-0.072746,-0.110348,-0.096082,-0.095219,-0.090929,0.0
3,-0.692654,0.348649,-0.415155,-0.513954,-0.604324,-0.520873,-0.561057,-0.555302,-0.511114,-0.570368,...,-0.354996,-0.328551,-0.206908,-0.189932,-0.072746,-0.110348,-0.096082,-0.095219,-0.090929,0.0
4,-0.704947,-0.628519,-0.100426,-0.576883,-0.386127,-0.464050,-0.570120,1.905627,-0.359078,0.034180,...,-0.354996,-0.328551,-0.206908,-0.189932,-0.072746,-0.110348,-0.096082,-0.095219,-0.090929,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6230,-0.704947,-0.262199,-0.117105,0.394724,-0.606233,-0.590130,0.271915,2.911910,-0.579097,-0.141946,...,-0.098807,-0.294427,-0.206908,-0.189932,-0.072746,-0.110348,-0.096082,-0.095219,-0.090929,0.0
6231,-0.704947,1.602956,-0.624208,-0.587951,-0.572107,0.652177,0.264001,-0.553661,-0.388661,-0.599890,...,-0.354996,-0.328551,-0.206908,-0.189932,-0.072746,-0.110348,-0.096082,-0.095219,-0.090929,0.0
6232,1.887094,-0.628519,-0.110167,1.678924,-0.502614,-0.458661,2.603447,-0.309895,-0.579097,0.097266,...,-0.354942,-0.328476,-0.206908,-0.189932,-0.072746,-0.110348,-0.096082,-0.095219,-0.090929,0.0
6233,-0.652399,-0.031896,-0.504382,0.014335,-0.268175,-0.501203,-0.075846,0.114062,-0.068623,-0.478769,...,-0.354996,-0.328551,-0.206908,-0.189932,-0.072746,-0.110348,-0.096082,-0.095219,-0.090929,0.0


In [176]:
x_temp = x_temp.T

In [177]:
trade_200_data = pd.merge(rank_rfe366,x_temp,on = rank_rfe366.index)
trade_200_data 

,key_0,rank,0,1,2,3,4,5,6,7,...,6225,6226,6227,6228,6229,6230,6231,6232,6233,6234
0,0,1,-0.529219,-0.581475,0.370049,-0.692654,-0.704947,-0.162719,-0.078751,1.097922,...,-0.467920,-0.704947,-0.704947,-0.579391,1.286610,-0.704947,-0.704947,1.887094,-0.652399,-0.700155
1,1,5,-0.628519,-0.628519,1.675549,0.348649,-0.628519,-0.165314,-0.606826,0.801930,...,0.149539,0.392886,-0.628519,-0.628519,-0.628519,-0.262199,1.602956,-0.628519,-0.031896,-0.397838
2,2,1,-0.624254,-0.580238,-0.560022,-0.415155,-0.100426,-0.026362,-0.584603,-0.553681,...,-0.599903,-0.332453,-0.624208,-0.064635,2.872816,-0.117105,-0.624208,-0.110167,-0.504382,-0.515776
3,3,2,-0.587999,-0.446588,-0.198879,-0.513954,-0.576883,0.503645,-0.164672,-0.587951,...,-0.357323,-0.518343,0.104792,3.246662,-0.568868,0.394724,-0.587951,1.678924,0.014335,-0.346398
4,4,4,-0.606277,1.638715,0.110310,-0.604324,-0.386127,-0.606233,2.128495,1.066129,...,-0.370684,-0.379603,-0.205382,2.052124,-0.562522,-0.606233,-0.572107,-0.502614,-0.268175,-0.581870
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,361,1,-0.110348,-0.110348,-0.110348,-0.110348,-0.110348,-0.110348,-0.110348,-0.110348,...,-0.110348,-0.110348,-0.110348,-0.110348,-0.110348,-0.110348,-0.110348,-0.110348,-0.110348,-0.110348
362,362,3,-0.096082,-0.096082,-0.096082,-0.096082,-0.096082,-0.096082,-0.096082,-0.096082,...,-0.096082,-0.096082,-0.096082,-0.096082,-0.096082,-0.096082,-0.096082,-0.096082,-0.096082,-0.096082
363,363,2,-0.095219,-0.095219,-0.095219,-0.095219,-0.095219,-0.095219,-0.095219,-0.095219,...,-0.095219,-0.095219,-0.095219,-0.095219,-0.095219,-0.095219,-0.095219,-0.095219,-0.095219,-0.095219
364,364,4,-0.090929,-0.090929,-0.090929,-0.090929,-0.090929,-0.090929,-0.090929,-0.090929,...,-0.090929,-0.090929,-0.090929,-0.090929,-0.090929,-0.090929,-0.090929,-0.090929,-0.090929,-0.090929


In [178]:
trade_200_data = trade_200_data.groupby(trade_200_data.index).filter(lambda x : float(x['rank'])==1)
trade_200_data

,key_0,rank,0,1,2,3,4,5,6,7,...,6225,6226,6227,6228,6229,6230,6231,6232,6233,6234
0,0,1,-0.529219,-0.581475,0.370049,-0.692654,-0.704947,-0.162719,-0.078751,1.097922,...,-0.467920,-0.704947,-0.704947,-0.579391,1.286610,-0.704947,-0.704947,1.887094,-0.652399,-0.700155
2,2,1,-0.624254,-0.580238,-0.560022,-0.415155,-0.100426,-0.026362,-0.584603,-0.553681,...,-0.599903,-0.332453,-0.624208,-0.064635,2.872816,-0.117105,-0.624208,-0.110167,-0.504382,-0.515776
5,5,1,-0.590168,3.719115,-0.590130,-0.520873,-0.464050,-0.487299,-0.386803,0.285427,...,-0.474073,3.592808,-0.393247,0.433325,-0.262569,-0.590130,0.652177,-0.458661,-0.501203,-0.590130
6,6,1,-0.590103,-0.459063,-0.590059,-0.561057,-0.570120,-0.590059,-0.571711,-0.069258,...,-0.291903,1.894442,-0.590059,-0.560305,-0.531480,0.271915,0.264001,2.603447,-0.075846,-0.590059
8,8,1,-0.579149,-0.536888,0.211710,-0.511114,-0.359078,-0.579097,-0.517792,-0.273564,...,-0.400400,2.436425,0.338337,-0.282329,1.163422,-0.579097,-0.388661,-0.579097,-0.068623,-0.561254
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350,350,1,-0.556283,-0.314873,-0.318330,-0.461838,-0.583441,-0.492228,0.039073,-0.635467,...,-0.354824,1.995403,-0.635512,-0.285157,-0.492228,-0.495190,-0.624245,-0.394281,2.755275,-0.599601
351,351,1,-0.099786,-0.530293,-0.494576,0.761507,-0.091892,0.216348,1.311873,-0.240134,...,0.195970,-0.496703,-0.536787,4.290094,0.216348,-0.426725,0.100016,-0.487242,1.324861,-0.492000
359,359,1,-0.189932,-0.051531,-0.189932,-0.189932,-0.189932,-0.189932,-0.189932,4.166090,...,-0.189932,-0.189932,-0.189932,-0.189932,-0.189932,-0.189932,-0.189932,-0.189932,-0.189932,-0.189932
360,360,1,-0.072746,-0.072746,-0.072746,-0.072746,-0.072746,-0.072746,-0.072746,-0.072746,...,-0.072746,-0.072746,-0.072746,-0.072746,-0.072746,-0.072746,-0.072746,-0.072746,-0.072746,-0.072746


In [181]:
trade_200_data = trade_200_data.drop(['key_0','rank'],axis=1)

In [183]:
trade_200_data = trade_200_data.T

In [184]:
trade_200_y_x = pd.merge(y_temp,trade_200_data,on = y_temp.index)

In [186]:
trade_200_y_x = trade_200_y_x.drop('key_0',axis = 1)
trade_200_y_x

,trade,0,2,5,6,8,9,10,13,14,...,341,344,345,346,349,350,351,359,360,361
0,0,-0.529219,-0.624254,-0.590168,-0.590103,-0.579149,-0.599936,-0.570282,-0.597530,-0.565185,...,-0.256554,-0.626634,2.017472,0.074835,-0.556920,-0.556283,-0.099786,-0.189932,-0.072746,-0.110348
1,0,-0.581475,-0.580238,3.719115,-0.459063,-0.536888,-0.594151,-0.556634,-0.313862,-0.242079,...,-0.471468,-0.608817,-0.425504,-0.249882,0.005915,-0.314873,-0.530293,-0.051531,-0.072746,-0.110348
2,0,0.370049,-0.560022,-0.590130,-0.590059,0.211710,0.012049,-0.570233,-0.562094,-0.498597,...,-0.515356,0.224663,-0.090935,-0.006478,-0.624776,-0.318330,-0.494576,-0.189932,-0.072746,-0.110348
3,0,-0.692654,-0.415155,-0.520873,-0.561057,-0.511114,-0.570368,-0.431785,-0.578538,-0.565185,...,-0.316017,0.902103,5.050156,-0.506619,1.336767,-0.461838,0.761507,-0.189932,-0.072746,-0.110348
4,0,-0.704947,-0.100426,-0.464050,-0.570120,-0.359078,0.034180,0.042440,2.720602,-0.565135,...,1.957665,-0.068958,-0.471799,-0.186443,2.397252,-0.583441,-0.091892,-0.189932,-0.072746,-0.110348
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6230,10,-0.704947,-0.117105,-0.590130,0.271915,-0.579097,-0.141946,-0.354194,-0.597483,-0.565135,...,-0.591688,-0.602451,-0.221561,1.400322,0.069095,-0.495190,-0.426725,-0.189932,-0.072746,-0.110348
6231,10,-0.704947,-0.624208,0.652177,0.264001,-0.388661,-0.599890,-0.081638,-0.597483,0.070282,...,-0.428425,1.862355,-0.487993,-0.431361,2.235098,-0.624245,0.100016,-0.189932,-0.072746,-0.110348
6232,10,1.887094,-0.110167,-0.458661,2.603447,-0.579097,0.097266,0.249847,0.221188,2.190374,...,-0.355975,-0.098440,-0.332026,-0.557751,-0.170044,-0.394281,-0.487242,-0.189932,-0.072746,-0.110348
6233,10,-0.652399,-0.504382,-0.501203,-0.075846,-0.068623,-0.478769,-0.570233,-0.557416,-0.446234,...,-0.591688,-0.616722,-0.477586,-0.102980,-0.523799,2.755275,1.324861,-0.189932,-0.072746,-0.110348


In [187]:
trade_200_y_x.to_csv('../data/predict_user_attribute20200911/raw_data/user_profile/trade/newdealspdata/trade_200_y_x.csv')

In [3]:
trade_200_y_x =pd.read_csv('../data/predict_user_attribute20200911/raw_data/user_profile/trade/newdealspdata/trade_200_y_x.csv')

In [9]:
x_data = trade_200_y_x.drop(['trade','Unnamed: 0'],axis = 1)
y_data = trade_200_y_x['trade']

In [10]:
trade_200_y_x

,Unnamed: 0,trade,0,2,5,6,8,9,10,13,...,341,344,345,346,349,350,351,359,360,361
0,0,0,-0.529219,-0.624254,-0.590168,-0.590103,-0.579149,-0.599936,-0.570282,-0.597530,...,-0.256554,-0.626634,2.017472,0.074835,-0.556920,-0.556283,-0.099786,-0.189932,-0.072746,-0.110348
1,1,0,-0.581475,-0.580238,3.719115,-0.459063,-0.536888,-0.594151,-0.556634,-0.313862,...,-0.471468,-0.608817,-0.425504,-0.249882,0.005915,-0.314873,-0.530293,-0.051531,-0.072746,-0.110348
2,2,0,0.370049,-0.560022,-0.590130,-0.590059,0.211710,0.012049,-0.570233,-0.562094,...,-0.515356,0.224663,-0.090935,-0.006478,-0.624776,-0.318330,-0.494576,-0.189932,-0.072746,-0.110348
3,3,0,-0.692654,-0.415155,-0.520873,-0.561057,-0.511114,-0.570368,-0.431785,-0.578538,...,-0.316017,0.902103,5.050156,-0.506619,1.336767,-0.461838,0.761507,-0.189932,-0.072746,-0.110348
4,4,0,-0.704947,-0.100426,-0.464050,-0.570120,-0.359078,0.034180,0.042440,2.720602,...,1.957665,-0.068958,-0.471799,-0.186443,2.397252,-0.583441,-0.091892,-0.189932,-0.072746,-0.110348
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6230,6230,10,-0.704947,-0.117105,-0.590130,0.271915,-0.579097,-0.141946,-0.354194,-0.597483,...,-0.591688,-0.602451,-0.221561,1.400322,0.069095,-0.495190,-0.426725,-0.189932,-0.072746,-0.110348
6231,6231,10,-0.704947,-0.624208,0.652177,0.264001,-0.388661,-0.599890,-0.081638,-0.597483,...,-0.428425,1.862355,-0.487993,-0.431361,2.235098,-0.624245,0.100016,-0.189932,-0.072746,-0.110348
6232,6232,10,1.887094,-0.110167,-0.458661,2.603447,-0.579097,0.097266,0.249847,0.221188,...,-0.355975,-0.098440,-0.332026,-0.557751,-0.170044,-0.394281,-0.487242,-0.189932,-0.072746,-0.110348
6233,6233,10,-0.652399,-0.504382,-0.501203,-0.075846,-0.068623,-0.478769,-0.570233,-0.557416,...,-0.591688,-0.616722,-0.477586,-0.102980,-0.523799,2.755275,1.324861,-0.189932,-0.072746,-0.110348


In [11]:
from sklearn.model_selection import train_test_split
x_train_data, x_test_data, y_train_data, y_test_data = train_test_split(x_data, y_data, random_state=155)

In [22]:
def svm(C):
    val = cross_val_score(
        SVC(C=C
           ),
        x_train_data, y_train_data, scoring='accuracy',cv=10).mean()
    return val
t_start = time.time()
params = {
    'C': (2,100)
    }
svm_bo = BayesianOptimization(svm,params)
svm_bo.maximize(init_points = 10,n_iter =20)
params_1 = svm_bo
params = svm_bo.max
params = params["params"]
L = list()
L.append(params)
val = svm(**params)
print(params)
print(val)
t_end = time.time()
print(t_end - t_start)
svm_classifier=SVC(C=L[0]['C'],kernel='linear')

# 拟合模型
svm_classifier.fit(x_train_data, y_train_data)

# 使用模型预测
preds = svm_classifier.predict(x_test_data)

print('准确率为%f' %((preds==y_test_data).sum()/float(y_test_data.shape[0])))
#joblib.dump(svm_classifier, '../code/model_svm')

|   iter    |  target   |     C     |
-------------------------------------
|  1        |  0.2423   |  36.86    |
|  2        |  0.2423   |  48.84    |
|  3        |  0.2423   |  32.1     |
|  4        |  0.2423   |  29.3     |
|  5        |  0.2423   |  80.97    |
|  6        |  0.2658   |  2.389    |
|  7        |  0.2423   |  29.48    |
|  8        |  0.256    |  3.737    |
|  9        |  0.2449   |  8.01     |
|  10       |  0.2423   |  54.95    |
|  11       |  0.2423   |  100.0    |
|  12       |  0.2707   |  2.001    |
|  13       |  0.271    |  2.0      |
|  14       |  0.2423   |  91.48    |
|  15       |  0.2423   |  91.48    |
|  16       |  0.271    |  2.006    |
|  17       |  0.2423   |  68.48    |
|  18       |  0.271    |  2.0      |
|  19       |  0.271    |  2.0      |
|  20       |  0.271    |  2.0      |
|  21       |  0.2712   |  2.0      |
|  22       |  0.271    |  2.0      |
|  23       |  0.271    |  2.001    |
|  24       |  0.2707   |  2.0      |
|  25       

SVC(C=2.000131041276405, break_ties=False, cache_size=200, class_weight=None,
    coef0=0.0, decision_function_shape='ovr', degree=3, gamma='scale',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

准确率为0.288005


In [206]:
sklearn.__version__

'0.23.2'

In [23]:
preds.tolist()

[1,
 1,
 1,
 1,
 1,
 4,
 0,
 1,
 4,
 4,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 4,
 1,
 3,
 1,
 7,
 1,
 1,
 1,
 4,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 4,
 4,
 1,
 1,
 1,
 4,
 1,
 1,
 1,
 1,
 1,
 1,
 4,
 4,
 1,
 1,
 4,
 3,
 1,
 1,
 4,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 4,
 4,
 1,
 1,
 1,
 10,
 1,
 1,
 1,
 1,
 1,
 1,
 4,
 1,
 4,
 4,
 1,
 1,
 1,
 1,
 4,
 4,
 1,
 3,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 7,
 1,
 1,
 4,
 4,
 1,
 4,
 1,
 4,
 1,
 1,
 1,
 4,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 4,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 4,
 4,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 8,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 4,
 6,
 1,
 1,
 1,
 1,
 3,
 1,
 1,
 1,
 1,
 1,
 1,
 4,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 6,
 4,
 1,
 1,
 1,
 4,
 1,
 1,
 1,
 1,
 1,
 8,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 4,
 1,
 1,
 4,
 1,
 1,
 1,
 1,
 4,
 4,
 1,
 1,
 4,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 4,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 4,
 1,
 1,
 1,
 0,
 4,
 4,

In [14]:
trade_200_data_none1 = trade_200_y_x.groupby(trade_200_y_x.index).filter(lambda x : float(x['trade'])!=1)

In [195]:
y_test_data.tolist()

[0,
 3,
 1,
 1,
 4,
 0,
 1,
 4,
 0,
 0,
 2,
 1,
 1,
 4,
 1,
 1,
 0,
 3,
 4,
 0,
 2,
 4,
 4,
 2,
 4,
 0,
 4,
 1,
 3,
 5,
 4,
 1,
 4,
 8,
 1,
 2,
 3,
 1,
 4,
 4,
 2,
 0,
 0,
 1,
 4,
 4,
 2,
 3,
 4,
 1,
 4,
 1,
 1,
 8,
 1,
 1,
 3,
 0,
 4,
 4,
 4,
 1,
 6,
 1,
 2,
 1,
 9,
 0,
 0,
 0,
 2,
 1,
 7,
 1,
 2,
 0,
 4,
 4,
 4,
 4,
 1,
 2,
 4,
 8,
 9,
 1,
 3,
 1,
 6,
 0,
 4,
 3,
 7,
 2,
 9,
 1,
 4,
 3,
 0,
 4,
 9,
 4,
 4,
 1,
 3,
 0,
 3,
 1,
 4,
 1,
 3,
 2,
 4,
 0,
 2,
 4,
 1,
 1,
 3,
 7,
 0,
 4,
 10,
 1,
 0,
 4,
 4,
 1,
 4,
 0,
 1,
 4,
 7,
 1,
 4,
 3,
 4,
 0,
 1,
 4,
 1,
 3,
 3,
 0,
 1,
 1,
 4,
 4,
 10,
 1,
 0,
 1,
 8,
 0,
 2,
 1,
 2,
 9,
 4,
 1,
 4,
 4,
 4,
 3,
 1,
 3,
 0,
 4,
 0,
 5,
 9,
 0,
 4,
 1,
 1,
 1,
 4,
 1,
 2,
 0,
 3,
 6,
 4,
 3,
 3,
 4,
 4,
 5,
 6,
 1,
 1,
 4,
 5,
 1,
 1,
 0,
 1,
 2,
 4,
 0,
 3,
 1,
 4,
 4,
 3,
 4,
 4,
 1,
 1,
 1,
 2,
 4,
 0,
 0,
 1,
 1,
 4,
 2,
 4,
 4,
 1,
 3,
 1,
 4,
 1,
 4,
 2,
 4,
 1,
 1,
 10,
 1,
 0,
 1,
 1,
 1,
 4,
 1,
 0,
 2,
 1,
 5,
 1,
 4,
 1,
 2,
 0,
 1,
 1,
 

In [19]:
trade_200_data_none1 = trade_200_data_none1.drop('Unnamed: 0',axis = 1)

In [24]:
preds1 = svm_classifier.predict(trade_200_data_none1)

In [25]:
preds.tolist()

[1,
 1,
 1,
 1,
 1,
 4,
 0,
 1,
 4,
 4,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 4,
 1,
 3,
 1,
 7,
 1,
 1,
 1,
 4,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 4,
 4,
 1,
 1,
 1,
 4,
 1,
 1,
 1,
 1,
 1,
 1,
 4,
 4,
 1,
 1,
 4,
 3,
 1,
 1,
 4,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 4,
 4,
 1,
 1,
 1,
 10,
 1,
 1,
 1,
 1,
 1,
 1,
 4,
 1,
 4,
 4,
 1,
 1,
 1,
 1,
 4,
 4,
 1,
 3,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 7,
 1,
 1,
 4,
 4,
 1,
 4,
 1,
 4,
 1,
 1,
 1,
 4,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 4,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 4,
 4,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 8,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 4,
 6,
 1,
 1,
 1,
 1,
 3,
 1,
 1,
 1,
 1,
 1,
 1,
 4,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 6,
 4,
 1,
 1,
 1,
 4,
 1,
 1,
 1,
 1,
 1,
 8,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 4,
 1,
 1,
 4,
 1,
 1,
 1,
 1,
 4,
 4,
 1,
 1,
 4,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 4,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 4,
 1,
 1,
 1,
 0,
 4,
 4,

In [19]:
df_datanone1 = df_data.groupby(df_data.index).filter(lambda x:float(x['trade'])!=1)

In [28]:
x_npa2 = x_npa1.T

In [31]:
x_npa2[300:700]

,0,1,2,3,4,5,6,7,8,9,...,6941,6942,6943,6944,6945,6946,6947,6948,6949,6950
300,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
301,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
302,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
303,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
304,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,7982,0,0,6762,1,821,0,674,12639,674,...,6253,1,549,0,0,6149,6519,1460,3957,3709
696,1706,0,0,58529,26600,23699,0,386,237,386,...,1,1620,1,0,0,11033,1236,2657,2095,1
697,759,0,0,41350,14454,233,0,534,705,534,...,31700,16275,1,0,0,3920,13010,2657,182,1
698,505,0,0,9034,273,32858,0,1069,49742,1069,...,1545,9043,8377,0,0,3742,1,0,1428,2775


In [76]:
ss = StandardScaler()
x_ss = ss.fit_transform(x_npa1)

In [61]:
from sklearn.decomposition import PCA
pca = PCA(n_components=100)   #降到200维
pca.fit(x_ss)                  #训练
x_data_pca=pca.fit_transform(x_ss) 

PCA(copy=True, iterated_power='auto', n_components=100, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [8]:
x_ss_df = pd.DataFrame(x_ss)

In [9]:
x_ss_df.columns = ["c{}".format(i) for i in range(5120)]

In [10]:
x_trade_data = x_ss_df.loc[:,["c{}".format(i) for i in range(35)]]

In [77]:
from sklearn.model_selection import train_test_split
x_train_data, x_test_data, y_train_data, y_test_data = train_test_split(x_ss, y_npa, random_state=155)

In [41]:
x_train_data

array([[-0.72255307,  0.03429136,  1.53414246, ..., -2.02372977,
        -2.72065184, -0.23038191],
       [-0.81722615, -0.17869794, -0.36749979, ..., -1.45406352,
         4.38478645,  2.21412009],
       [-0.87997189,  0.28380432,  1.20075846, ..., -0.9763793 ,
        -1.64236725,  0.82560335],
       ...,
       [ 0.85585077, -0.45843896, -1.29471495, ...,  0.60791286,
         2.28706226, -1.25991385],
       [-0.16201676, -0.18612779, -2.23344017, ...,  1.01633338,
        -1.8030207 , -1.2917742 ],
       [-1.52234142,  4.13747222,  4.22234379, ..., -2.54029271,
         0.78829598, -1.28027075]])

In [ ]:
def svm(C):
    val = cross_val_score(
        SVC(C=C
           ),
        x_train_data, y_train_data, scoring='accuracy',cv=10).mean()
    return val
t_start = time.time()
params = {
    'C': (0.001,100)
    }
svm_bo = BayesianOptimization(svm,params)
svm_bo.maximize(init_points = 10,n_iter =20)
params_1 = svm_bo
params = svm_bo.max
params = params["params"]
L = list()
L.append(params)
val = svm(**params)
print(params)
print(val)
t_end = time.time()
print(t_end - t_start)
svm_classifier=SVC(C=L[0]['C'],kernel='linear')

# 拟合模型
svm_classifier.fit(x_train_data, y_train_data)

# 使用模型预测
preds = svm_classifier.predict(x_test_data)

print('准确率为%f' %((preds==y_test_data).sum()/float(y_test_data.shape[0])))
#joblib.dump(svm_classifier, '../code/model_svm')

|   iter    |  target   |     C     |
-------------------------------------
|  1        |  0.2484   |  43.05    |
|  2        |  0.2484   |  65.88    |


In [58]:
preds

array([1, 1, 1, ..., 1, 1, 1], dtype=uint8)

In [28]:
joblib.dump(svm_classifier, '../code/trademodel_svm/trademodel_svm')

['../code/trademodel_svm/trademodel_svm']

In [64]:
model_trade = joblib.load('../code/model_svm_sum/model_svm1')

/home/zourui/anaconda3/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator SVC from version 0.23.2 when using version 0.22.2. This might lead to breaking code or invalid results. Use at your own risk.
  state = self.__dict__.copy()


In [65]:
model_trade.predict(x_test_data)

AttributeError: 'SVC' object has no attribute 'probA_'

In [ ]:
x_npa = x_npa.reshape(len(x_npa),5120)
x_npa = pd.DataFrame(x_npa)
pca = PCA(n_components=200)   #降到200维
#    pca.fit(x_npa)                  #训练
df_datapca_x=pca.fit_transform(x_npa) 

In [30]:
preds.tolist()

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [31]:
pred_test = svm_classifier.predict(df_datapca_x)

In [32]:
temp = pd.DataFrame(pred_test)

In [33]:
temp[50:100]

,0
50,1
51,1
52,1
53,1
54,1
55,1
56,1
57,1
58,1
59,1


In [21]:
df_data_x = df_datanone1.drop('trade',axis = 1)
from sklearn.decomposition import PCA
pca = PCA(n_components=200)   #降到200维
pca.fit(df_data_x)                  #训练
df_datapca_x=pca.fit_transform(df_data_x) 

PCA(copy=True, iterated_power='auto', n_components=200, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [34]:
scikit-learn.__version__

NameError: name 'scikit' is not defined

In [45]:
preds.tolist()
#y_test_data

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 4,
 1,
 1,
 1,
 1,
 4,
 4,
 1,
 1,
 1,
 1,
 4,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 4,
 1,
 1,
 1,
 4,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 4,
 1,
 1,
 1,
 4,
 1,
 1,
 1,
 1,
 1,
 4,
 1,
 1,
 1,
 1,
 1,
 1,
 4,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 4,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 4,
 4,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 4,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 4,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 4,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 4,
 4,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 4,
 1,
 1,
 1,
 1,
 1,
 4,
 1,
 1,
 4,
 4,
 1,
 1,
 4,
 1,
 1,
 3,
 1,
 1,
 1,
 4,
 1,
 1,
 1,
 4,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 4,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 4,


In [2]:
df_trade = pd.read_csv(r'../data/predict_user_attribute20200911/raw_data/user_profile/trade/trade_svm.csv')

In [3]:
df_trade

,Unnamed: 0,trade,c0,c1,c2,c3,c4,c5,c6,c7,...,c5110,c5111,c5112,c5113,c5114,c5115,c5116,c5117,c5118,c5119
0,0,3,326,327,1403,2230,3252,11,7852,234,...,24,754,4892,11,5306,392,2418,7910,7852,11
1,1,3,2964,11,358,6724,464,1,129,0,...,31,40016,2820,114494,7196,3539,62,11519,4859,38
2,2,3,1,8,16274,37167,55,535,12286,57,...,88,29667,1699,179,129,91029,18,1,8,1581
3,3,3,1,16593,233,0,31305,1518,21201,2,...,44659,8820,35451,514,10529,10956,8,1808,11,179
4,4,3,1343,1344,414,12148,128,111031,0,31305,...,39173,8,2812,8949,67,11,1,82,1,514
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1842,1842,10,1,483,109374,495,58307,11,101786,483,...,112,48954,112,1,111,140,129,281,454,8
1843,1843,10,1,2,82743,6929,325,378,1346,48005,...,572,4966,4780,11,1041,5201,26078,512,9451,3015
1844,1844,10,62203,245,1,0,9802,2,1,82,...,336,1985,12638,18,141,119,9573,13527,874,11
1845,1845,10,1262,112,3533,112,27163,1270,4760,1,...,894,1,1,362,15922,12890,11,14402,22917,1


In [4]:
df_trade = df_trade.drop('Unnamed: 0',axis=1)
df_trade

,trade,c0,c1,c2,c3,c4,c5,c6,c7,c8,...,c5110,c5111,c5112,c5113,c5114,c5115,c5116,c5117,c5118,c5119
0,3,326,327,1403,2230,3252,11,7852,234,7000,...,24,754,4892,11,5306,392,2418,7910,7852,11
1,3,2964,11,358,6724,464,1,129,0,7143,...,31,40016,2820,114494,7196,3539,62,11519,4859,38
2,3,1,8,16274,37167,55,535,12286,57,0,...,88,29667,1699,179,129,91029,18,1,8,1581
3,3,1,16593,233,0,31305,1518,21201,2,56952,...,44659,8820,35451,514,10529,10956,8,1808,11,179
4,3,1343,1344,414,12148,128,111031,0,31305,1518,...,39173,8,2812,8949,67,11,1,82,1,514
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1842,10,1,483,109374,495,58307,11,101786,483,1,...,112,48954,112,1,111,140,129,281,454,8
1843,10,1,2,82743,6929,325,378,1346,48005,0,...,572,4966,4780,11,1041,5201,26078,512,9451,3015
1844,10,62203,245,1,0,9802,2,1,82,1,...,336,1985,12638,18,141,119,9573,13527,874,11
1845,10,1262,112,3533,112,27163,1270,4760,1,2,...,894,1,1,362,15922,12890,11,14402,22917,1


In [38]:
df_trade.groupby('trade').count()

,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9,...,c5110,c5111,c5112,c5113,c5114,c5115,c5116,c5117,c5118,c5119
trade,,,,,,,,,,,,,,,,,,,,,
0,200,200,200,200,200,200,200,200,200,200,...,200,200,200,200,200,200,200,200,200,200
1,300,300,300,300,300,300,300,300,300,300,...,300,300,300,300,300,300,300,300,300,300
2,180,180,180,180,180,180,180,180,180,180,...,180,180,180,180,180,180,180,180,180,180
3,180,180,180,180,180,180,180,180,180,180,...,180,180,180,180,180,180,180,180,180,180
4,200,200,200,200,200,200,200,200,200,200,...,200,200,200,200,200,200,200,200,200,200
5,149,149,149,149,149,149,149,149,149,149,...,149,149,149,149,149,149,149,149,149,149
6,162,162,162,162,162,162,162,162,162,162,...,162,162,162,162,162,162,162,162,162,162
7,97,97,97,97,97,97,97,97,97,97,...,97,97,97,97,97,97,97,97,97,97
8,85,85,85,85,85,85,85,85,85,85,...,85,85,85,85,85,85,85,85,85,85


In [41]:
temp1 = df_trade.groupby(df_trade.index).filter(lambda x:float(x['trade'])==1)
temp_1 = df_trade.groupby(df_trade.index).filter(lambda x:float(x['trade'])!=1)

In [44]:
temp_r1 = temp1[0:200]
new_df = pd.concat([temp_r1, temp_1], axis=0, ignore_index=True)

In [45]:
new_df.groupby('trade').count()

,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9,...,c5110,c5111,c5112,c5113,c5114,c5115,c5116,c5117,c5118,c5119
trade,,,,,,,,,,,,,,,,,,,,,
0,200,200,200,200,200,200,200,200,200,200,...,200,200,200,200,200,200,200,200,200,200
1,200,200,200,200,200,200,200,200,200,200,...,200,200,200,200,200,200,200,200,200,200
2,180,180,180,180,180,180,180,180,180,180,...,180,180,180,180,180,180,180,180,180,180
3,180,180,180,180,180,180,180,180,180,180,...,180,180,180,180,180,180,180,180,180,180
4,200,200,200,200,200,200,200,200,200,200,...,200,200,200,200,200,200,200,200,200,200
5,149,149,149,149,149,149,149,149,149,149,...,149,149,149,149,149,149,149,149,149,149
6,162,162,162,162,162,162,162,162,162,162,...,162,162,162,162,162,162,162,162,162,162
7,97,97,97,97,97,97,97,97,97,97,...,97,97,97,97,97,97,97,97,97,97
8,85,85,85,85,85,85,85,85,85,85,...,85,85,85,85,85,85,85,85,85,85


In [86]:
new_df_x = new_df.loc[:,["c{}".format(i) for i in range(500)]]

In [87]:
ss = StandardScaler()
x_ss = ss.fit_transform(new_df_x)

In [88]:
y_data = new_df['trade']

In [89]:
from sklearn.model_selection import train_test_split
x_train_data, x_test_data, y_train_data, y_test_data = train_test_split(x_ss, y_data, random_state=155)

In [91]:
estimator = SVC(kernel = "linear")
selector = RFE(estimator,n_features_to_select = 200,step = 1)
selector = selector.fit(x_train_data,y_train_data)
rank_rfe = selector.ranking_

In [92]:
rank_rfe = pd.DataFrame(rank_rfe)

In [93]:
x_train_data = pd.DataFrame(x_train_data)
x_train_data

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,0.691340,-0.508053,-0.450700,0.311731,-0.516221,-0.501159,0.111999,-0.502987,-0.296142,-0.499762,...,0.957066,-0.413595,-0.417626,-0.285315,0.470272,1.031688,-0.415577,0.257101,1.313182,-0.353165
1,-0.333154,-0.450520,-0.472538,-0.383733,0.427636,0.687090,-0.476599,-0.373206,-0.502299,-0.506076,...,0.100917,-0.045066,-0.408650,-0.186525,0.874554,1.351002,-0.415796,4.036527,0.119966,-0.352968
2,-0.487580,0.412361,-0.182114,1.774012,4.757370,-0.500469,-0.281161,-0.359117,-0.491458,-0.472591,...,-0.129040,-0.417035,-0.176896,0.145829,-0.405310,0.034387,-0.047439,-0.310657,-0.317075,-0.353165
3,-0.314456,-0.327595,0.229839,-0.012448,-0.519365,-0.445130,-0.353999,-0.448854,-0.508053,-0.185657,...,-0.380894,-0.417527,-0.418085,-0.326551,-0.320690,-0.384187,-0.378945,-0.061659,-0.157378,-0.353165
4,-0.576911,-0.479902,-0.185985,-0.334329,-0.429324,0.546769,1.322249,-0.501808,-0.510112,1.025104,...,4.106752,-0.412121,-0.402228,-0.424075,-0.395968,-0.234431,0.370659,-0.273552,-0.067699,-0.345424
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1305,-0.492922,-0.400904,-0.129909,-0.239150,-0.382961,-0.501033,1.268642,0.293192,-0.509627,-0.506020,...,-0.235894,-0.400466,-0.209723,-0.417687,-0.348919,-0.417439,-0.243342,0.286309,-0.353608,-0.345227
1306,-0.177442,-0.510633,3.691331,1.423226,-0.344543,2.791280,-0.006874,-0.510761,1.385817,-0.420503,...,-0.256631,-0.354970,0.535007,-0.286200,-0.408221,-0.094742,-0.413684,-0.325759,-0.268342,-0.250948
1307,-0.591256,3.945870,-0.461770,-0.473228,-0.500843,-0.500469,1.329040,-0.507013,-0.086170,4.835693,...,-0.381513,-0.417597,-0.418806,-0.424771,-0.408965,-0.426680,-0.416378,-0.392608,-0.375152,-0.353690
1308,-0.593284,-0.511220,-0.468848,-0.473228,-0.519022,-0.353192,3.521949,6.368228,-0.491156,1.016480,...,4.457104,0.297555,-0.415071,-0.424265,-0.408898,-0.191939,-0.165855,-0.389618,-0.375022,-0.350082


In [94]:
x_test_data = pd.DataFrame(x_test_data)
x_test_data = x_test_data.T

In [95]:
x_train_data = x_train_data.T

In [96]:
rank_rfe.columns = ['rank']
new_rank_test200 = pd.merge(rank_rfe,x_test_data,on=rank_rfe.index)
new_rank_test200 = new_rank_test200.drop('key_0',axis = 1)
new_rank_test200

,rank,0,1,2,3,4,5,6,7,8,...,427,428,429,430,431,432,433,434,435,436
0,1,-0.592937,-0.545353,-0.314456,-0.491982,-0.592888,-0.515477,0.026003,-0.595559,-0.508305,...,0.537359,-0.255792,0.494128,-0.579681,2.638828,0.090009,-0.595559,-0.402848,0.459701,-0.500242
1,1,-0.413044,-0.011426,-0.327595,-0.511220,0.247737,-0.508053,-0.510809,-0.482952,1.797966,...,-0.496910,0.536224,0.504085,-0.383837,3.597974,-0.511220,-0.304077,-0.233407,0.308379,-0.226604
2,1,2.243205,-0.472538,0.229839,-0.469271,4.692963,1.101594,-0.405996,-0.472538,-0.447010,...,2.805481,0.534841,0.573919,-0.450700,-0.472114,-0.467638,-0.465823,-0.333587,-0.240489,-0.143701
3,1,-0.465488,-0.473168,-0.012448,4.201694,-0.469842,-0.469842,0.903670,-0.443356,1.485873,...,-0.473289,-0.278939,-0.462465,-0.453817,-0.433983,-0.473228,-0.050000,-0.377928,-0.473228,-0.357670
4,1,-0.461968,-0.211397,-0.519365,-0.464026,-0.519422,1.637670,-0.518851,-0.518851,-0.519365,...,1.270189,-0.403027,0.784597,0.033171,-0.519480,0.572558,-0.516907,-0.064073,-0.518851,0.267334
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,1,-0.386333,-0.425195,-0.384187,-0.343675,-0.189298,-0.425773,0.621777,-0.426515,4.370958,...,0.916751,4.281434,-0.417439,-0.426020,0.336952,-0.425773,0.727555,-0.426515,0.301637,-0.426598
496,88,-0.382004,-0.251499,-0.378945,-0.394822,-0.416305,-0.150343,-0.411499,-0.037097,2.160881,...,-0.044380,-0.415577,-0.324325,2.117549,-0.415577,-0.142914,-0.416305,-0.412373,4.917221,-0.416305
497,59,-0.356194,-0.388468,-0.061659,-0.208236,-0.391995,0.723434,-0.391765,-0.391765,-0.391995,...,-0.391765,0.480723,0.032712,-0.187231,-0.382565,-0.373903,-0.382719,-0.389772,0.138275,-0.324226
498,164,-0.373984,-0.273533,-0.157378,-0.286706,-0.371583,-0.374438,2.871409,-0.375087,-0.311883,...,-0.289042,0.162405,0.124314,0.532608,-0.372751,-0.371583,-0.369442,0.149751,-0.253027,-0.366781


In [97]:
df_tradetest_rfe = new_rank_test200.groupby(new_rank_test200.index).filter(lambda x:float(x['rank'])==1)
df_tradetest_rfe = df_tradetest_rfe.drop('rank',axis = 1)
df_tradetest_rfe = df_tradetest_rfe.T
df_tradetest_rfe

,0,1,2,3,4,5,6,7,8,11,...,463,464,468,469,479,484,485,493,494,495
0,-0.592937,-0.413044,2.243205,-0.465488,-0.461968,-0.460422,-0.473774,-0.510691,-0.506660,-0.516725,...,-0.409559,-0.177866,-0.410765,-0.331894,-0.399817,-0.339427,-0.345733,-0.368166,-0.408424,-0.386333
1,-0.545353,-0.011426,-0.472538,-0.473168,-0.211397,-0.500532,-0.388916,-0.394512,-0.490429,-0.536849,...,-0.312128,0.606983,-0.407780,0.283584,-0.365848,-0.387882,-0.253852,-0.424265,-0.251437,-0.425195
2,-0.314456,-0.327595,0.229839,-0.012448,-0.519365,-0.445130,-0.353999,-0.448854,-0.508053,0.317584,...,-0.151134,-0.352606,0.939563,-0.388128,-0.320246,-0.378396,-0.364823,-0.326551,-0.320690,-0.384187
3,-0.491982,-0.511220,-0.469271,4.201694,-0.464026,-0.497586,-0.477079,0.169518,-0.509991,1.343003,...,-0.386295,0.167617,7.147242,-0.401490,-0.400250,-0.387882,-0.232853,-0.403204,0.413475,-0.343675
4,-0.592888,0.247737,4.692963,-0.469842,-0.519422,-0.497712,-0.477019,-0.506805,-0.510112,-0.536849,...,5.249487,-0.404770,-0.415831,-0.328429,-0.397404,1.359811,-0.355610,-0.421166,-0.396848,-0.189298
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
432,0.090009,-0.511220,-0.467638,-0.473228,0.572558,-0.396247,-0.291257,-0.376953,-0.509446,-0.503682,...,-0.404323,0.981739,1.995980,-0.407738,-0.400250,2.001021,-0.325232,-0.324653,0.097673,-0.425773
433,-0.595559,-0.304077,-0.465823,-0.050000,-0.516907,0.448939,-0.433148,-0.510761,-0.051346,-0.530265,...,-0.387355,-0.314111,0.249721,0.055185,4.315287,0.082709,-0.365653,-0.424707,-0.408221,0.727555
434,-0.402848,-0.233407,-0.333587,-0.377928,-0.064073,-0.500657,-0.477019,-0.488482,-0.506781,1.129162,...,3.804849,-0.407929,-0.407711,-0.408171,-0.140376,-0.380692,-0.346065,2.822816,-0.375591,-0.426515
435,0.459701,0.308379,-0.240489,-0.473228,-0.518851,-0.153458,3.728145,-0.125024,-0.490671,-0.528899,...,-0.409559,2.027859,-0.415345,-0.065387,-0.380636,-0.338883,-0.365404,-0.424707,-0.362458,0.301637


In [98]:
rank_rfe.columns = ['rank']
new_rank200 = pd.merge(rank_rfe,x_train_data,on=rank_rfe.index)
new_rank200 = new_rank200.drop('key_0',axis = 1)
new_rank200

,rank,0,1,2,3,4,5,6,7,8,...,1300,1301,1302,1303,1304,1305,1306,1307,1308,1309
0,1,0.691340,-0.333154,-0.487580,-0.314456,-0.576911,-0.541149,-0.592888,0.450254,0.728784,...,-0.595559,-0.177343,-0.595559,-0.362931,0.012153,-0.492922,-0.177442,-0.591256,-0.593284,-0.531009
1,1,-0.508053,-0.450520,0.412361,-0.327595,-0.479902,-0.430404,-0.489110,-0.450520,1.059066,...,-0.511220,-0.509284,-0.504710,-0.294517,-0.473509,-0.400904,-0.510633,3.945870,-0.511220,1.080473
2,1,-0.450700,-0.472538,-0.182114,0.229839,-0.185985,-0.469271,-0.326509,-0.469331,-0.274728,...,-0.471933,-0.472114,-0.258879,-0.370608,1.097480,-0.129909,3.691331,-0.461770,-0.468848,-0.472538
3,1,0.311731,-0.383733,1.774012,-0.012448,-0.334329,3.149700,5.625879,-0.157455,-0.108898,...,-0.473228,0.568485,-0.466516,-0.467363,-0.003559,-0.239150,1.423226,-0.473228,-0.473228,-0.472805
4,1,-0.516221,0.427636,4.757370,-0.519365,-0.429324,-0.516221,0.768476,-0.503815,-0.157201,...,0.559867,-0.514792,0.596798,-0.519422,0.726457,-0.382961,-0.344543,-0.500843,-0.519022,-0.518851
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,1,1.031688,1.351002,0.034387,-0.384187,-0.234431,-0.426680,-0.426185,-0.426598,-0.421152,...,1.815122,1.691934,0.177790,-0.426515,-0.426020,-0.417439,-0.094742,-0.426680,-0.191939,-0.396811
496,88,-0.415577,-0.415796,-0.047439,-0.378945,0.370659,-0.416378,-0.391180,-0.415067,4.520025,...,-0.415577,-0.384844,-0.416305,-0.396060,-0.411280,-0.243342,-0.413684,-0.416378,-0.165855,-0.397735
497,59,0.257101,4.036527,-0.310657,-0.061659,-0.273552,-0.392608,-0.370836,1.105593,1.333202,...,0.148854,-0.391765,-0.391228,-0.185391,-0.391228,0.286309,-0.325759,-0.392608,-0.389618,-0.328979
498,164,1.313182,0.119966,-0.317075,-0.157378,-0.067699,-0.375152,-0.374633,-0.374438,-0.374438,...,-0.372751,-0.317140,-0.249653,0.525340,0.469404,-0.353608,-0.268342,-0.375152,-0.375022,0.107183


In [99]:
df_trade_rfe = new_rank200.groupby(new_rank200.index).filter(lambda x:float(x['rank'])==1)
df_trade_rfe = df_trade_rfe.drop('rank',axis = 1)
df_trade_rfe = df_trade_rfe.T
df_trade_rfe

,0,1,2,3,4,5,6,7,8,11,...,463,464,468,469,479,484,485,493,494,495
0,0.691340,-0.508053,-0.450700,0.311731,-0.516221,-0.501159,0.111999,-0.502987,-0.296142,-0.529955,...,1.817380,-0.407736,2.501980,4.313857,-0.400436,1.183574,-0.346065,-0.285315,0.470272,1.031688
1,-0.333154,-0.450520,-0.472538,-0.383733,0.427636,0.687090,-0.476599,-0.373206,-0.502299,-0.442629,...,0.093770,-0.400127,-0.399591,-0.383921,-0.378532,-0.387882,-0.329050,-0.186525,0.874554,1.351002
2,-0.487580,0.412361,-0.182114,1.774012,4.757370,-0.500469,-0.281161,-0.359117,-0.491458,-0.536911,...,0.104110,-0.400127,-0.312355,-0.407120,0.190097,-0.326317,-0.330461,0.145829,-0.405310,0.034387
3,-0.314456,-0.327595,0.229839,-0.012448,-0.519365,-0.445130,-0.353999,-0.448854,-0.508053,0.317584,...,-0.151134,-0.352606,0.939563,-0.388128,-0.320246,-0.378396,-0.364823,-0.326551,-0.320690,-0.384187
4,-0.576911,-0.479902,-0.185985,-0.334329,-0.429324,0.546769,1.322249,-0.501808,-0.510112,-0.536787,...,-0.403528,0.409416,-0.415345,0.409043,-0.391773,-0.386130,0.219246,-0.424075,-0.395968,-0.234431
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1305,-0.492922,-0.400904,-0.129909,-0.239150,-0.382961,-0.501033,1.268642,0.293192,-0.509627,-0.407973,...,-0.405848,-0.055354,-0.408821,2.308866,0.565120,-0.387882,0.385162,-0.417687,-0.348919,-0.417439
1306,-0.177442,-0.510633,3.691331,1.423226,-0.344543,2.791280,-0.006874,-0.510761,1.385817,-0.536787,...,-0.380065,0.012156,-0.356077,-0.407120,-0.400869,-0.350302,6.705584,-0.286200,-0.408221,-0.094742
1307,-0.591256,3.945870,-0.461770,-0.473228,-0.500843,-0.500469,1.329040,-0.507013,-0.086170,-0.516725,...,-0.410288,-0.408445,-0.416594,-0.408233,-0.400931,-0.388365,-0.366317,-0.424771,-0.408965,-0.426680
1308,-0.593284,-0.511220,-0.468848,-0.473228,-0.519022,-0.353192,3.521949,6.368228,-0.491156,-0.174195,...,7.594802,-0.408316,-0.407642,-0.404398,7.072117,-0.251641,0.623205,-0.424265,-0.408898,-0.191939


In [ ]:
x_test_data

In [100]:
def svm(C):
    val = cross_val_score(
        SVC(C=C
           ),
        df_trade_rfe, y_train_data, scoring='accuracy',cv=10).mean()
    return val
t_start = time.time()
params = {
    'C': (0.0001,1)
    }
svm_bo = BayesianOptimization(svm,params)
svm_bo.maximize(init_points = 10,n_iter =20)
params_1 = svm_bo
params = svm_bo.max
params = params["params"]
L = list()
L.append(params)
val = svm(**params)
print(params)
print(val)
t_end = time.time()
print(t_end - t_start)
svm_classifier=SVC(C=L[0]['C'],kernel = 'linear')

# 拟合模型
svm_classifier.fit(df_trade_rfe, y_train_data)

# 使用模型预测
preds = svm_classifier.predict(df_tradetest_rfe)

print('准确率为%f' %((preds==y_test_data).sum()/float(y_test_data.shape[0])))

|   iter    |  target   |     C     |
-------------------------------------
|  1        |  0.1374   |  0.6172   |
|  2        |  0.1366   |  0.5004   |
|  3        |  0.1176   |  0.222    |
|  4        |  0.1183   |  0.3578   |
|  5        |  0.1374   |  0.5555   |
|  6        |  0.1336   |  0.7984   |
|  7        |  0.1389   |  0.5332   |
|  8        |  0.1366   |  0.5384   |
|  9        |  0.1336   |  0.8027   |
|  10       |  0.1191   |  0.02334  |
|  11       |  0.1313   |  1.0      |
|  12       |  0.1336   |  0.9094   |
|  13       |  0.1374   |  0.5908   |
|  14       |  0.1389   |  0.5745   |
|  15       |  0.1382   |  0.5759   |
|  16       |  0.1389   |  0.5721   |
|  17       |  0.1382   |  0.6337   |
|  18       |  0.1374   |  0.5824   |
|  19       |  0.1382   |  0.5757   |
|  20       |  0.1397   |  0.6513   |
|  21       |  0.1412   |  0.6781   |
|  22       |  0.1389   |  0.7009   |
|  23       |  0.1405   |  0.6885   |
|  24       |  0.1405   |  0.6905   |
|  25       

SVC(C=0.6780929152848135, break_ties=False, cache_size=200, class_weight=None,
    coef0=0.0, decision_function_shape='ovr', degree=3, gamma='scale',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

准确率为0.098398


In [101]:
preds

array([ 3,  1,  1,  4,  0,  0,  0,  6,  5,  0,  1,  1,  5,  3,  9,  2,  1,
        6,  6,  8, 10,  2,  4,  6,  0,  0,  0,  3,  7,  6,  4,  9,  0, 10,
        3,  8,  3,  4, 10,  5,  1,  6,  3,  1,  4,  5,  3,  4,  8,  4,  1,
       10,  7,  1,  1,  5,  9,  5,  2,  1,  2,  1,  1,  9,  2,  3,  0,  5,
        5,  1,  0,  6,  4,  1,  1,  4,  6,  5,  6,  4,  4,  5,  8,  4,  1,
        6,  1,  2,  6,  8, 10,  2,  4,  6,  9,  2,  6,  0,  6,  6,  2,  5,
        2,  3,  5,  3,  6,  2,  2,  6,  2,  0,  1,  1,  9,  2,  0,  2,  3,
        0,  1,  2,  6,  5,  1,  5,  8,  4,  7,  0,  0,  1,  7,  8,  3,  2,
        0,  3,  3,  3,  4,  6,  6,  0,  3,  6, 10,  0,  6,  9,  6,  5,  4,
        6,  1,  0,  1,  8,  5,  7,  2, 10,  3,  0,  1,  9,  4,  1,  7,  2,
        4,  2,  7,  6,  1, 10,  5,  7,  8,  4,  0,  3,  1,  1,  1,  2, 10,
        4,  6,  0,  2,  3,  2,  6,  5,  1,  4,  4,  5,  3,  3,  2,  2,  1,
        4,  3,  2,  3,  6,  2,  4,  9,  7,  7,  5,  1,  3,  5,  6,  1,  6,
        0,  5,  6,  1, 10

In [80]:
def svm(C,gamma):
    val = cross_val_score(
        SVC(C=C,
            gamma = gamma
           ),
        df_trade_rfe, y_train_data, scoring='accuracy',cv=10).mean()
    return val
t_start = time.time()
params = {
    'C': (0.0001,1),
    'gamma':(0.0001,0.1)
    }
svm_bo = BayesianOptimization(svm,params)
svm_bo.maximize(init_points = 10,n_iter =20)
params_1 = svm_bo
params = svm_bo.max
params = params["params"]
L = list()
L.append(params)
val = svm(**params)
print(params)
print(val)
t_end = time.time()
print(t_end - t_start)
svm_classifier=SVC(C=L[0]['C'],gamma = L[0]['gamma'],kernel = 'rbf')

# 拟合模型
svm_classifier.fit(df_trade_rfe, y_train_data)

# 使用模型预测
preds_rbf = svm_classifier.predict(df_tradetest_rfe)

print('准确率为%f' %((preds==y_test_data).sum()/float(y_test_data.shape[0])))

|   iter    |  target   |     C     |   gamma   |
-------------------------------------------------
|  1        |  0.1328   |  0.4789   |  0.01174  |
|  2        |  0.1191   |  0.2071   |  0.02927  |
|  3        |  0.1252   |  0.652    |  0.03792  |
|  4        |  0.1252   |  0.9292   |  0.02905  |
|  5        |  0.1237   |  0.6394   |  0.09741  |
|  6        |  0.1252   |  0.877    |  0.01629  |
|  7        |  0.1244   |  0.598    |  0.07471  |
|  8        |  0.129    |  0.7473   |  0.02601  |
|  9        |  0.1229   |  0.9908   |  0.0429   |
|  10       |  0.1206   |  0.8256   |  0.04915  |
|  11       |  0.1191   |  0.4744   |  0.000122 |
|  12       |  0.1191   |  0.0001   |  0.1      |
|  13       |  0.1191   |  0.2944   |  0.1      |
|  14       |  0.1191   |  0.0001   |  0.0001   |
|  15       |  0.1221   |  0.4416   |  0.1      |
|  16       |  0.1191   |  0.122    |  0.1      |
|  17       |  0.1191   |  1.0      |  0.0001   |
|  18       |  0.1176   |  0.379    |  0.04236  |


SVC(C=0.4789340042778004, break_ties=False, cache_size=200, class_weight=None,
    coef0=0.0, decision_function_shape='ovr', degree=3,
    gamma=0.01173552429664934, kernel='rbf', max_iter=-1, probability=False,
    random_state=None, shrinking=True, tol=0.001, verbose=False)

准确率为0.118993


In [81]:
preds

array([ 3,  1,  1,  4,  1,  0,  1,  1,  2,  1,  1,  0,  0,  0,  1,  2,  1,
        0,  0,  0,  0,  2,  4,  0,  0,  0,  0,  0,  2,  0,  4,  1,  0,  1,
        0,  1,  1,  1,  1,  1,  1,  0,  0,  1,  0,  0,  0,  1,  5,  4,  1,
        0,  0,  1,  1,  1,  0,  4,  2,  0,  1,  1,  0,  0,  2,  3,  0,  1,
        0,  1,  0,  6,  4,  0,  1,  0,  1,  0,  1,  4,  1,  1,  1,  0,  1,
        1,  1,  1,  6,  0,  1,  4,  0,  0,  0,  1,  1,  0,  0,  0,  0,  0,
        2,  3,  0,  0,  0,  0,  1,  0,  0,  0,  1,  1,  1,  2,  0,  2,  0,
        0,  0,  0,  0,  0,  0,  1,  0,  4,  1,  0,  0,  1,  0,  1,  3,  4,
        1,  3,  1,  0,  1,  0,  0,  0,  0,  6,  0,  0,  1,  0,  1,  0,  4,
        1,  1,  4,  0,  0,  0,  0,  0,  1,  1,  0,  1,  0,  4,  1,  1,  0,
        0,  2,  0,  6,  1,  1,  0,  1,  1,  4,  1,  1,  1,  1,  1,  2,  1,
        1,  0,  0,  1,  1,  1,  6,  0,  0,  0,  1,  1,  0,  3,  1,  1,  1,
        4,  3,  2,  3,  1,  4,  0,  1,  0,  4,  1,  1,  0,  0,  6,  1,  0,
        0,  0,  0,  1,  0

In [32]:
y_test_data.tolist()

[3,
 1,
 8,
 4,
 3,
 6,
 5,
 5,
 0,
 10,
 0,
 4,
 10,
 5,
 10,
 4,
 4,
 0,
 2,
 2,
 7,
 3,
 1,
 10,
 10,
 4,
 3,
 1,
 0,
 0,
 5,
 8,
 5,
 3,
 4,
 0,
 8,
 9,
 9,
 5,
 6,
 5,
 3,
 1,
 0,
 5,
 0,
 0,
 4,
 5,
 5,
 7,
 6,
 1,
 2,
 3,
 1,
 3,
 5,
 1,
 0,
 1,
 5,
 1,
 10,
 2,
 10,
 1,
 10,
 1,
 2,
 3,
 0,
 2,
 5,
 6,
 2,
 8,
 1,
 1,
 1,
 2,
 0,
 1,
 2,
 5,
 1,
 6,
 3,
 9,
 1,
 1,
 10,
 1,
 2,
 7,
 3,
 9,
 9,
 2,
 5,
 0,
 6,
 0,
 7,
 7,
 9,
 3,
 6,
 3,
 3,
 0,
 7,
 9,
 9,
 10,
 9,
 1,
 4,
 8,
 8,
 7,
 9,
 4,
 10,
 6,
 3,
 6,
 3,
 5,
 4,
 9,
 1,
 8,
 8,
 5,
 4,
 4,
 3,
 9,
 8,
 1,
 5,
 2,
 2,
 2,
 6,
 2,
 3,
 0,
 9,
 1,
 10,
 10,
 10,
 5,
 8,
 2,
 0,
 0,
 10,
 3,
 7,
 5,
 2,
 2,
 1,
 4,
 1,
 4,
 9,
 7,
 0,
 2,
 10,
 10,
 3,
 2,
 6,
 10,
 1,
 8,
 3,
 9,
 6,
 4,
 0,
 4,
 3,
 1,
 4,
 3,
 5,
 9,
 7,
 3,
 8,
 4,
 0,
 8,
 3,
 2,
 9,
 8,
 3,
 4,
 1,
 10,
 2,
 2,
 4,
 10,
 1,
 3,
 1,
 3,
 0,
 3,
 8,
 6,
 2,
 3,
 1,
 10,
 1,
 10,
 8,
 7,
 3,
 3,
 4,
 0,
 6,
 9,
 3,
 4,
 0,
 4,
 1,
 1,
 3,
 7,
 1,
 6,
 0

In [83]:
y_train_data

250     3
1057    5
278     3
129     1
450     2
       ..
69      1
608     0
151     1
377     3
948     4
Name: trade, Length: 1310, dtype: int64

In [84]:
df_trade_rfe = pd.DataFrame(df_trade_rfe)
df_tradetest_rfe = pd.DataFrame(df_tradetest_rfe)
y_train_data = pd.DataFrame(y_train_data)
y_test_data = pd.DataFrame(y_test_data)
train_trade = pd.merge(y_train_data,df_trade_rfe,on = y_train_data.index)
test_trade = pd.merge(y_test_data,df_tradetest_rfe,on = y_test_data.index)
train_trade.groupby('trade').count()
test_trade.groupby('trade').count()

,key_0,1,2,3,4,5,6,7,10,13,...,271,273,279,281,286,288,289,294,296,297
trade,,,,,,,,,,,,,,,,,,,,,
0,156,156,156,156,156,156,156,156,156,156,...,156,156,156,156,156,156,156,156,156,156
1,149,149,149,149,149,149,149,149,149,149,...,149,149,149,149,149,149,149,149,149,149
2,137,137,137,137,137,137,137,137,137,137,...,137,137,137,137,137,137,137,137,137,137
3,138,138,138,138,138,138,138,138,138,138,...,138,138,138,138,138,138,138,138,138,138
4,149,149,149,149,149,149,149,149,149,149,...,149,149,149,149,149,149,149,149,149,149
5,112,112,112,112,112,112,112,112,112,112,...,112,112,112,112,112,112,112,112,112,112
6,124,124,124,124,124,124,124,124,124,124,...,124,124,124,124,124,124,124,124,124,124
7,68,68,68,68,68,68,68,68,68,68,...,68,68,68,68,68,68,68,68,68,68
8,60,60,60,60,60,60,60,60,60,60,...,60,60,60,60,60,60,60,60,60,60


,key_0,1,2,3,4,5,6,7,10,13,...,271,273,279,281,286,288,289,294,296,297
trade,,,,,,,,,,,,,,,,,,,,,
0,44,44,44,44,44,44,44,44,44,44,...,44,44,44,44,44,44,44,44,44,44
1,51,51,51,51,51,51,51,51,51,51,...,51,51,51,51,51,51,51,51,51,51
2,43,43,43,43,43,43,43,43,43,43,...,43,43,43,43,43,43,43,43,43,43
3,42,42,42,42,42,42,42,42,42,42,...,42,42,42,42,42,42,42,42,42,42
4,51,51,51,51,51,51,51,51,51,51,...,51,51,51,51,51,51,51,51,51,51
5,37,37,37,37,37,37,37,37,37,37,...,37,37,37,37,37,37,37,37,37,37
6,38,38,38,38,38,38,38,38,38,38,...,38,38,38,38,38,38,38,38,38,38
7,29,29,29,29,29,29,29,29,29,29,...,29,29,29,29,29,29,29,29,29,29
8,25,25,25,25,25,25,25,25,25,25,...,25,25,25,25,25,25,25,25,25,25


In [107]:
    #建模模型
    #strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])
    #with strategy.scope():
model = build_model(
    input_number = input_number,
    sentence_maxlen = sentence_maxlen,
    vocab_size = vocab_size,
    tag_size = tag_size,
    embedding_dim = embedding_dim,
    embedding_matrix = embedding_matrix,
    is_embedding_training = is_embedding_training,
    embedding_dropout_rate = embedding_dropout_rate,
    learning_rate = learning_rate
)

    #model_callback = build_model_callback()

print(model.summary())

history = model.fit(
    {"input{}".format(i):x_train_eval[:,i] for i in range(input_number)},
    y_train_eval,
    validation_split = 0.25,
    batch_size=batch_size,
    epochs=epochs,
    verbose=1,
        #callbacks=list(model_callback),
 #       class_weight = class_weight_dict,
)

    #测试集
model.evaluate({"input{}".format(i):x_test[:,i] for i in range(input_number)},y_test)
    #保存模型
print("即将保存模型：{}".format(model_path))
#model.save(model_path)

这是一个多分类模型
Model: "multi_textcnn"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input0 (InputLayer)             [(None, 512)]        0                                            
__________________________________________________________________________________________________
input1 (InputLayer)             [(None, 512)]        0                                            
__________________________________________________________________________________________________
input2 (InputLayer)             [(None, 512)]        0                                            
__________________________________________________________________________________________________
input3 (InputLayer)             [(None, 512)]        0                                            
____________________________________________________________________________

[25.330509185791016, 0.17613227665424347]

即将保存模型：/home/zourui/code/model_output_trade


In [108]:
np.shape(x_npa)

(6951, 10, 512)

In [109]:
x_npa = x_npa.reshape(6951,5120)

In [110]:
x_npa = pd.DataFrame(x_npa)
x_npa

,0,1,2,3,4,5,6,7,8,9,...,5110,5111,5112,5113,5114,5115,5116,5117,5118,5119
0,4218,19901,29015,9975,36996,3292,0,3005,21140,7978,...,0,0,0,0,0,0,0,0,0,0
1,1,2229,7807,796,0,31305,10361,5558,875,7752,...,0,0,0,0,0,0,0,0,0,0
2,4738,13789,0,770,5337,4738,1330,28126,3192,1330,...,0,0,0,0,0,0,0,0,0,0
3,5394,2085,0,27113,908,18090,63532,2333,1,20677,...,0,0,0,0,0,0,0,0,0,0
4,1,15973,1,61543,0,31305,21201,1,8935,7752,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6946,1,21794,0,31305,21201,1,7752,1,9802,28247,...,0,0,0,0,0,0,0,0,0,0
6947,5342,407,77351,0,5342,407,6645,39952,14786,31305,...,0,0,0,0,0,0,0,0,0,0
6948,9310,21856,14417,1,0,31305,21201,41546,7752,8788,...,0,0,0,0,0,0,0,0,0,0
6949,1307,4364,7492,1653,0,31305,10361,7752,1,9802,...,0,0,0,0,0,0,0,0,0,0


In [113]:
x_npa = x_npa.T


,0,1,2,3,4,5,6,7,8,9,...,6941,6942,6943,6944,6945,6946,6947,6948,6949,6950
0,4218,1,4738,5394,1,1,70644,2964,41348,1,...,39678,1342,1016,4965,65866,1,5342,9310,1307,2459
1,19901,2229,13789,2085,15973,1,1,1,82594,1,...,23849,1391,8523,2270,13758,21794,407,21856,4364,238
2,29015,7807,0,0,1,27163,1024,0,12371,409,...,88305,18311,1,3001,20674,0,77351,14417,7492,7827
3,9975,796,770,27113,61543,0,0,7143,3132,0,...,1,14352,5389,1336,1,31305,0,1,1653,14919
4,36996,0,5337,908,0,7143,1,27162,12726,1,...,0,0,10,0,5065,21201,5342,0,0,903
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [133]:
x_data_trade = x_npa1[0:200]

In [134]:
x_data_trade = x_data_trade.T
x_data_trade

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,4218,19901,29015,9975,36996,3292,0,3005,21140,7978,...,0,0,0,0,0,0,0,0,0,0
1,1,2229,7807,796,0,31305,10361,5558,875,7752,...,1,1,6713,1,12137,2739,26445,30206,4458,565
2,4738,13789,0,770,5337,4738,1330,28126,3192,1330,...,0,0,0,0,0,0,0,0,0,0
3,5394,2085,0,27113,908,18090,63532,2333,1,20677,...,0,0,0,0,0,0,0,0,0,0
4,1,15973,1,61543,0,31305,21201,1,8935,7752,...,32562,4929,12407,254,9476,21366,1,30969,5282,3010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6946,1,21794,0,31305,21201,1,7752,1,9802,28247,...,126,12411,0,0,0,0,0,0,0,0
6947,5342,407,77351,0,5342,407,6645,39952,14786,31305,...,0,0,0,0,0,0,0,0,0,0
6948,9310,21856,14417,1,0,31305,21201,41546,7752,8788,...,25,635,413,184,6071,659,12137,2559,50,267
6949,1307,4364,7492,1653,0,31305,10361,7752,1,9802,...,0,0,0,0,0,0,0,0,0,0


In [135]:
ss = StandardScaler()
x_ss = ss.fit_transform(x_data_trade)
from sklearn.model_selection import train_test_split
x_train_data, x_test_data, y_train_data, y_test_data = train_test_split(x_ss, y_npa, random_state=155)

In [ ]:
def svm(C):
    val = cross_val_score(
        SVC(C=C
           ),
        x_train_data, y_train_data, scoring='accuracy',cv=10).mean()
    return val
t_start = time.time()
params = {
    'C': (500,1000)
    }
svm_bo = BayesianOptimization(svm,params)
svm_bo.maximize(init_points = 10,n_iter =20)
params_1 = svm_bo
params = svm_bo.max
params = params["params"]
L = list()
L.append(params)
val = svm(**params)
print(params)
print(val)
t_end = time.time()
print(t_end - t_start)
svm_classifier=SVC(C=L[0]['C'],kernel = 'linear')

# 拟合模型
svm_classifier.fit(x_train_data, y_train_data)

# 使用模型预测
preds = svm_classifier.predict(x_test_data)

print('准确率为%f' %((preds==y_test_data).sum()/float(y_test_data.shape[0])))

|   iter    |  target   |     C     |
-------------------------------------
|  1        |  0.234    |  574.3    |
|  2        |  0.2338   |  622.7    |
|  3        |  0.234    |  655.6    |
|  4        |  0.234    |  613.0    |
|  5        |  0.2337   |  820.7    |
|  6        |  0.2333   |  744.2    |
|  7        |  0.2342   |  640.9    |
|  8        |  0.2337   |  667.7    |
|  9        |  0.234    |  608.4    |
|  10       |  0.234    |  614.8    |
|  11       |  0.2342   |  1e+03    |
|  12       |  0.2342   |  1e+03    |
|  13       |  0.2342   |  1e+03    |
|  14       |  0.2342   |  1e+03    |
|  15       |  0.2342   |  1e+03    |
|  16       |  0.2342   |  999.9    |
|  17       |  0.2342   |  999.9    |
|  18       |  0.2342   |  1e+03    |
|  19       |  0.2342   |  999.9    |
|  20       |  0.2338   |  500.0    |
|  21       |  0.2342   |  999.8    |
|  22       |  0.2342   |  999.7    |
|  23       |  0.2342   |  1e+03    |
|  24       |  0.2342   |  999.5    |
|  25       

In [140]:
preds.tolist()

[4,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 6,
 0,
 4,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 4,
 1,
 4,
 1,
 4,
 1,
 1,
 4,
 7,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 4,
 1,
 1,
 1,
 1,
 4,
 1,
 1,
 10,
 1,
 4,
 4,
 0,
 4,
 1,
 1,
 4,
 1,
 4,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 4,
 1,
 4,
 1,
 1,
 1,
 4,
 1,
 1,
 4,
 1,
 1,
 3,
 1,
 3,
 1,
 4,
 1,
 3,
 1,
 1,
 4,
 3,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 7,
 3,
 4,
 1,
 4,
 4,
 1,
 1,
 1,
 1,
 1,
 1,
 3,
 1,
 1,
 1,
 1,
 1,
 1,
 4,
 1,
 1,
 4,
 1,
 1,
 1,
 4,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 4,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 3,
 1,
 4,
 1,
 4,
 1,
 1,
 1,
 4,
 1,
 1,
 1,
 1,
 1,
 3,
 1,
 1,
 1,
 1,
 4,
 1,
 1,
 1,
 8,
 4,
 1,
 1,
 4,
 1,
 7,
 1,
 1,
 1,
 1,
 4,
 4,
 1,
 1,
 1,
 4,
 1,
 9,
 1,
 1,
 4,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 4,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 4,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 3,
 1,
 3,
 1,